# Presencia web de negocios locales
## Introducción
#### En este notebook se realizo la consolidación y limpieza de datos de negocios locales obtenidos de Apify Google Maps Scraper.
**Objetivos:**
  Reunir datos de distintas ciudades colindantes del centro de la provincia de Buenos Aires que estén dentro del rango entre 16000 y 160000 habitantes.
- Estandarizar las bases de datos.
- Eliminar información innecesaria.
- Generar un archivo consolidado limpio para el análisis exploratorio posterior.


**Datos del censo de 2022**
- Tandil : 150.162 habitantes
- Azul : 75.905 habitantes
- Olavarria : 125.751 habitantes
- Benito Juárez : 22.292 habitantes
- Rauch : 16.635 habitantes


# Data Preparation - Presencia web de negocios locales

In [2]:
# Se importan las librerías para la carga y manipulación de los datos.
# pandas: manejo y limpieza de datos
# numpy: operaciones numéricas
# glob, os: manipulación de archivos y carpetas

import pandas as pd
import os
import numpy as np
from glob import glob
import glob

# Configura pandas para mostrar todas las columnas del DataFrame al mostrarlo, sin recorte visual.
pd.set_option('display.max_columns', None) 


In [3]:
# Carga de archivo CSV
df_azul_1= pd.read_csv('../data/raw/azul_dataset_google-maps-extractor_2025-05-19_00-03-31-957.csv')

In [4]:
# Vemos las dimensiones del dataframe
df_azul_1.shape

(233, 161)

In [5]:
# El método info() nos muestra un resumen de la estructura del dataframe, una rápida inspección de los datos (información sobre las columnas, tipos de datos, 
# valores no nulos y uso de memoria) y para identificar posibles problemas, como datos faltantes o tipos de datos incorrectos
df_azul_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Columns: 161 entries, additionalInfo/Accesibilidad/0/Entrada accesible para personas en silla de ruedas to website
dtypes: bool(4), float64(12), int64(2), object(142), uint64(1)
memory usage: 286.8+ KB


In [6]:
# Se listan las columnas del DataFrame
df_azul_1.columns

Index(['additionalInfo/Accesibilidad/0/Entrada accesible para personas en silla de ruedas',
       'additionalInfo/Accesibilidad/0/Estacionamiento accesible para personas en silla de ruedas',
       'additionalInfo/Accesibilidad/1/Espacio accesible para personas en silla de ruedas',
       'additionalInfo/Accesibilidad/1/Estacionamiento accesible para personas en silla de ruedas',
       'additionalInfo/Accesibilidad/2/Estacionamiento accesible para personas en silla de ruedas',
       'additionalInfo/Accesibilidad/3/Sanitarios accesibles para personas en silla de ruedas',
       'additionalInfo/Ambiente/0/A la moda',
       'additionalInfo/Ambiente/1/Agradable',
       'additionalInfo/Ambiente/2/Informal',
       'additionalInfo/Ambiente/3/Relajado',
       ...
       'scrapedAt', 'searchPageUrl', 'searchString', 'state', 'street',
       'temporarilyClosed', 'title', 'totalScore', 'url', 'website'],
      dtype='object', length=161)

In [7]:
# Selección de columnas relevantes
df_azul_1 = df_azul_1[[
        'title',
        'categoryName',
        'website',
        'street',
        'phone',
        'city',
        'state',
        'location/lat', 
        'location/lng',
        'totalScore',
        'rank',
        'imageUrl',
        'imagesCount',       
        'reviewsCount', 
        'scrapedAt',
        'searchString']]
df_azul_1.head(100)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,La Casa de los Trofeos,Comercio,NaN,Rivadavia 496,NaN,Azul,Provincia de Buenos Aires,-36.774661,-59.862623,NaN,1,NaN,NaN,0,2025-05-19T00:03:00.547Z,Comercio
1,La Escoba Loca,Tienda de artículos para el hogar,NaN,Colón Nte. 117,+54 2281 50-2996,Azul,Provincia de Buenos Aires,-36.767281,-59.851848,4.8,1,https://lh3.googleusercontent.com/gps-cs-s/AC9...,44.0,73,2025-05-19T00:03:02.157Z,Comercio
2,Artículos magali,Tienda general,NaN,Monseñor Caneva Nte. 160,+54 2281 31-0611,Azul,Provincia de Buenos Aires,-36.774353,-59.841898,4.3,2,NaN,NaN,12,2025-05-19T00:03:02.157Z,Comercio
3,Autoservicio San Jorge,Comercio,NaN,Burgos Nte. 100,+54 2281 42-3119,Azul,Provincia de Buenos Aires,-36.768145,-59.851305,4.6,3,https://lh3.googleusercontent.com/p/AF1QipPV0V...,6.0,127,2025-05-19T00:03:02.157Z,Comercio
4,Asociacion Latina de Exportacion SA,Comercio,NaN,Julio Argentino Roca 846,+54 2281 43-0338,Azul,Provincia de Buenos Aires,-36.785067,-59.859254,3.3,4,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,4,2025-05-19T00:03:02.158Z,Comercio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Distribuidora FERRE-COR-,Tienda de herramientas,http://distriuidoraferrecor.com/,Miñana 763,+54 3543 51-0978,Azul,Provincia de Buenos Aires,-36.790486,-59.867821,5.0,65,https://lh3.googleusercontent.com/p/AF1QipN30q...,13.0,1,2025-05-19T00:03:07.002Z,Comercio
96,Al Sur Del Atardecer,Frutería,NaN,Av. 25 de Mayo 1579,NaN,Azul,Provincia de Buenos Aires,-36.792780,-59.875774,4.3,66,https://lh3.googleusercontent.com/gps-cs-s/AC9...,18.0,44,2025-05-19T00:03:07.002Z,Comercio
97,Kolman,Tienda de materiales para la construcción,NaN,"nuevo telefono 02281 42-4971, Av. Piazza 1142",+54 2281 42-4971,Azul,Provincia de Buenos Aires,-36.785359,-59.848930,4.4,67,https://lh3.googleusercontent.com/gps-cs-s/AC9...,3.0,16,2025-05-19T00:03:07.002Z,Comercio
98,Tienda boliviana Meyli,Tienda de ropa,NaN,San Martín 990,+54 2281 47-4537,Azul,Provincia de Buenos Aires,-36.784963,-59.854366,NaN,68,https://lh3.googleusercontent.com/gps-cs-s/AC9...,5.0,0,2025-05-19T00:03:07.002Z,Comercio


In [8]:
# Información del DataFrame sececcionado a través del método .info() : 
# Número de filas y columnas
# Nombres y tipos de datos de las columnas
# Valores no nulos en cada columna
# Uso de memoria
df_azul_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         233 non-null    object 
 1   categoryName  232 non-null    object 
 2   website       32 non-null     object 
 3   street        201 non-null    object 
 4   phone         112 non-null    object 
 5   city          233 non-null    object 
 6   state         233 non-null    object 
 7   location/lat  233 non-null    float64
 8   location/lng  233 non-null    float64
 9   totalScore    172 non-null    float64
 10  rank          233 non-null    int64  
 11  imageUrl      220 non-null    object 
 12  imagesCount   219 non-null    float64
 13  reviewsCount  233 non-null    int64  
 14  scrapedAt     233 non-null    object 
 15  searchString  233 non-null    object 
dtypes: float64(4), int64(2), object(10)
memory usage: 29.3+ KB


In [9]:
# Se guarda DataFrame en CSV
df_azul_1.to_csv('../data/processed/01_dataPrep/azul/01_Azul.csv', index = False)

**Se reiteran procedimientos con todos los archivos .CSV de la misma ciudad**

In [10]:
df_azul_2 = pd.read_csv('../data/raw/azul_dataset_crawler-google-places_2025-06-15_22-59-32-816.csv')

In [11]:
df_azul_2.shape

(286, 428)

In [12]:
# Seleccionaos las 16 columnas que necesitamos del dataframe
df_azul_2 = df_azul_2[[
        'title',
        'categoryName',
        'website',
        'street',
        'phone',
        'city',
        'state',
        'location/lat', 
        'location/lng',
        'totalScore',
        'rank',
        'imageUrl',
        'imagesCount',       
        'reviewsCount', 
        'scrapedAt',
        'searchString']]
df_azul_2.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,DTI - Azul,Soporte y servicios informáticos,NaN,Av. Juan Domingo Perón 525,+54 2281 42-7066,Azul,Provincia de Buenos Aires,-36.782006,-59.865497,NaN,22,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,0,2025-06-15T22:56:55.477Z,tecnologia
1,TECNO COMPUTACION,Tienda de móviles,http://www.tecnocomputacionazul.com/,Lamadrid 33,+54 2281 65-7938,Azul,Provincia de Buenos Aires,-36.764914,-59.862404,4.6,3,https://lh3.googleusercontent.com/p/AF1QipP9Gg...,12.0,7,2025-06-15T22:56:54.474Z,tecnologia
2,Ingeniero Marcelo Edgardo Cornec,Ingeniero,NaN,Av. Juan Domingo Perón 622,+54 2281 42-6984,Azul,Provincia de Buenos Aires,-36.783278,-59.863594,5.0,20,https://lh3.googleusercontent.com/p/AF1QipN3mw...,13.0,3,2025-06-15T22:56:57.478Z,comercio
3,La esquina,Kiosco,NaN,"7300, Gral. Paz 1500",+54 2281 36-0736,Azul,Provincia de Buenos Aires,-36.798090,-59.853525,3.7,19,https://lh3.googleusercontent.com/p/AF1QipP1o8...,1.0,3,2025-06-15T22:56:57.478Z,comercio
4,Vidrieria 25 de Mayo de Jose Canalicchio,Comercio,NaN,Guaminí 610,+54 2281 53-7036,Azul,Provincia de Buenos Aires,-36.785552,-59.867145,4.2,40,https://lh3.googleusercontent.com/p/AF1QipM1o-...,5.0,9,2025-06-15T22:56:58.824Z,comercio


In [13]:
# Se revisa la estructura para verificar las 16 columnas elegidas
df_azul_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         286 non-null    object 
 1   categoryName  277 non-null    object 
 2   website       75 non-null     object 
 3   street        262 non-null    object 
 4   phone         204 non-null    object 
 5   city          283 non-null    object 
 6   state         283 non-null    object 
 7   location/lat  286 non-null    float64
 8   location/lng  286 non-null    float64
 9   totalScore    226 non-null    float64
 10  rank          286 non-null    int64  
 11  imageUrl      279 non-null    object 
 12  imagesCount   279 non-null    float64
 13  reviewsCount  286 non-null    int64  
 14  scrapedAt     286 non-null    object 
 15  searchString  286 non-null    object 
dtypes: float64(4), int64(2), object(10)
memory usage: 35.9+ KB


In [14]:
# Guardamos el dataframe en un archivo .CSV y utilizamos index=False para omitir la información del index
df_azul_2.to_csv('../data/processed/01_dataPrep/azul/02_Azul.csv', index = False)

In [15]:
# Búsqueda de los archivos .csv en el directorio
glob.glob('../data/processed/01_dataPrep/azul/*.csv')

['../data/processed/01_dataPrep/azul/02_Azul.csv',
 '../data/processed/01_dataPrep/azul/01_Azul.csv']

In [16]:
# Lectura y unión de archivos CSV
# Lectura de todos los archivos CSV de la carpeta y se combinan en un único DataFrame con `pd.concat()`. Se listan las columnas del DataFrame resultante.

df = pd.concat(map(pd.read_csv, glob.glob('../data/processed/01_dataPrep/azul/*.csv')))
df.columns

Index(['title', 'categoryName', 'website', 'street', 'phone', 'city', 'state',
       'location/lat', 'location/lng', 'totalScore', 'rank', 'imageUrl',
       'imagesCount', 'reviewsCount', 'scrapedAt', 'searchString'],
      dtype='object')

In [17]:
# Vizualizamos las primeras 5 filas del dataframe
df.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,DTI - Azul,Soporte y servicios informáticos,NaN,Av. Juan Domingo Perón 525,+54 2281 42-7066,Azul,Provincia de Buenos Aires,-36.782006,-59.865497,NaN,22,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,0,2025-06-15T22:56:55.477Z,tecnologia
1,TECNO COMPUTACION,Tienda de móviles,http://www.tecnocomputacionazul.com/,Lamadrid 33,+54 2281 65-7938,Azul,Provincia de Buenos Aires,-36.764914,-59.862404,4.6,3,https://lh3.googleusercontent.com/p/AF1QipP9Gg...,12.0,7,2025-06-15T22:56:54.474Z,tecnologia
2,Ingeniero Marcelo Edgardo Cornec,Ingeniero,NaN,Av. Juan Domingo Perón 622,+54 2281 42-6984,Azul,Provincia de Buenos Aires,-36.783278,-59.863594,5.0,20,https://lh3.googleusercontent.com/p/AF1QipN3mw...,13.0,3,2025-06-15T22:56:57.478Z,comercio
3,La esquina,Kiosco,NaN,"7300, Gral. Paz 1500",+54 2281 36-0736,Azul,Provincia de Buenos Aires,-36.798090,-59.853525,3.7,19,https://lh3.googleusercontent.com/p/AF1QipP1o8...,1.0,3,2025-06-15T22:56:57.478Z,comercio
4,Vidrieria 25 de Mayo de Jose Canalicchio,Comercio,NaN,Guaminí 610,+54 2281 53-7036,Azul,Provincia de Buenos Aires,-36.785552,-59.867145,4.2,40,https://lh3.googleusercontent.com/p/AF1QipM1o-...,5.0,9,2025-06-15T22:56:58.824Z,comercio


In [18]:
# Se guarda el archivo principal de la ciudad
df.to_csv('../data/processed/01_dataPrep/main/main_azul.csv', index = False)

**Se repiten procedimientos con los archivos raw de otras ciudades**

In [19]:
# Lectura del archivo .CSV
df_juarez_01 = pd.read_csv('../data/raw/juarez_dataset_crawler-google-places_2025-05-29_00-54-32-651.csv')
df_juarez_01.head(5)

,additionalInfo/Accesibilidad/0/Bucle magnético de asistencia,additionalInfo/Accesibilidad/0/Entrada accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/2/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Ambiente/0/A la moda,additionalInfo/Ambiente/0/Agradable,additionalInfo/Ambiente/1/Agradable,additionalInfo/Ambiente/1/Informal,additionalInfo/Ambiente/2/Informal,additionalInfo/Ambiente/2/Relajado,additionalInfo/Ambiente/3/Relajado,additionalInfo/Aspectos destacados/0/Asientos en la terraza,additionalInfo/Aspectos destacados/0/Buen café,additionalInfo/Aspectos destacados/0/Buenos productos frescos,additionalInfo/Aspectos destacados/0/Deliciosos postres,additionalInfo/Aspectos destacados/1/Buen café,additionalInfo/Aspectos destacados/1/Deliciosos postres,additionalInfo/Aspectos destacados/1/Muy buena selección de cervezas,additionalInfo/Aspectos destacados/2/Chimenea,additionalInfo/Aspectos destacados/3/Deliciosos postres,additionalInfo/Aspectos destacados/4/Música en vivo,additionalInfo/Aspectos destacados/5/Muy buena selección de cervezas,additionalInfo/Estacionamiento/0/Estacionamiento en el lugar,additionalInfo/Estacionamiento/0/Estacionamiento gratuito en la calle,additionalInfo/Estacionamiento/1/Estacionamiento gratuito,additionalInfo/Estacionamiento/2/Estacionamiento gratuito en la calle,additionalInfo/Estacionamiento/3/Garage de stacionamiento gratuito,additionalInfo/Información proporcionada por la empresa/0/Se identifica como mujer empresaria,additionalInfo/Mascotas/0/Se permiten perros,additionalInfo/Menores/0/Ideal para ir con niños,additionalInfo/Menores/1/Menú para niños,additionalInfo/Menores/1/Sillas altas,additionalInfo/Menores/2/Sillas altas,additionalInfo/Opciones de servicio/0/Asientos al aire libre,additionalInfo/Opciones de servicio/0/Citas en línea,additionalInfo/Opciones de servicio/0/Compras en tienda,additionalInfo/Opciones de servicio/0/Consumo en el lugar,additionalInfo/Opciones de servicio/0/Entrega a domicilio,additionalInfo/Opciones de servicio/0/Entrega el mismo día,additionalInfo/Opciones de servicio/0/Estimaciones en línea,additionalInfo/Opciones de servicio/0/Retiro en tienda,additionalInfo/Opciones de servicio/0/Retiros en la puerta,additionalInfo/Opciones de servicio/0/Servicios en el lugar,additionalInfo/Opciones de servicio/1/Citas en línea,additionalInfo/Opciones de servicio/1/Compras en tienda,additionalInfo/Opciones de servicio/1/Entrega a domicilio,additionalInfo/Opciones de servicio/1/Entrega el mismo día,additionalInfo/Opciones de servicio/1/Entrega sin contacto,additionalInfo/Opciones de servicio/1/Para llevar,additionalInfo/Opciones de servicio/1/Retiro en tienda,additionalInfo/Opciones de servicio/1/Servicios en el lugar,additionalInfo/Opciones de servicio/2/Compras en tienda,additionalInfo/Opciones de servicio/2/Consumo en el lugar,additionalInfo/Opciones de servicio/2/Entrega a domicilio,additionalInfo/Opciones de servicio/2/Entrega el mismo día,additionalInfo/Opciones de servicio/2/Para llevar,additionalInfo/Opciones de servicio/2/Retiro en tienda,additionalInfo/Opciones de servicio/2/Servicios en el lugar,additionalInfo/Opciones de servicio/3/Compras en tienda,additionalInfo/Opciones de servicio/3/Entrega a domicilio,additionalInfo/Opciones de servicio/3/Entrega el mismo día,additionalInfo/Opciones de servicio/3/Para llevar,additionalInfo/Opciones de servicio/4/Consumo en el lugar,additionalInfo/Opciones de servicio/4/Entrega el 

In [20]:
df_juarez_01.shape

(256, 251)

In [21]:
# Conservamos las columnas necesarias
df_juarez_01 = df_juarez_01[[
        'title',
        'categoryName',
        'website',
        'street',
        'phone',
        'city',
        'state',
        'location/lat', 
        'location/lng',
        'totalScore',
        'rank',
        'imageUrl',
        'imagesCount',       
        'reviewsCount', 
        'scrapedAt',
        'searchString']]
df_juarez_01.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Serantes Accesorios,Tienda de repuestos de automóviles usados,NaN,Av. Muñiz 49,+54 2281 46-6108,Benito Juárez,Provincia de Buenos Aires,-37.678436,-59.800652,4.6,1,https://lh3.googleusercontent.com/gps-cs-s/AC9...,2.0,66,2025-05-29T00:53:21.747Z,repuestos
1,Carlos REYNOSO REPUESTOS AGRICOLAS,Tienda de repuestos para coches de carreras,NaN,Av. Muñiz 251-299,NaN,Benito Juárez,Provincia de Buenos Aires,-37.682303,-59.805742,4.9,2,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,11,2025-05-29T00:53:21.748Z,repuestos
2,Repuestos lo de migue,Tienda de accesorios para automóviles,NaN,San Juan 126,+54 2281 53-2680,Benito Juárez,Provincia de Buenos Aires,-37.672441,-59.800303,4.6,3,https://lh3.googleusercontent.com/gps-cs-s/AC9...,4.0,17,2025-05-29T00:53:21.748Z,repuestos
3,Repuestos Perco,Tienda de repuestos de automóviles usados,NaN,Av. Urquiza 41,+54 2281 53-2090,Benito Juárez,Provincia de Buenos Aires,-37.672897,-59.803693,4.6,4,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,5,2025-05-29T00:53:21.749Z,repuestos
4,Acoplados Juarez - Repuestos acoplados y semir...,Proveedor de repuestos de carrocería de automó...,https://www.guia-dorada.com.ar/guia/aviso-repu...,"Ruta Provincial 86, Suipacha y",+54 9 249 463-4600,Benito Juárez,Provincia de Buenos Aires,-37.679447,-59.818685,5.0,5,https://lh3.googleusercontent.com/gps-cs-s/AC9...,26.0,2,2025-05-29T00:53:21.749Z,repuestos


In [22]:
# Obtenemos la información del Dataframe
df_juarez_01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         256 non-null    object 
 1   categoryName  252 non-null    object 
 2   website       52 non-null     object 
 3   street        236 non-null    object 
 4   phone         156 non-null    object 
 5   city          254 non-null    object 
 6   state         254 non-null    object 
 7   location/lat  256 non-null    float64
 8   location/lng  256 non-null    float64
 9   totalScore    164 non-null    float64
 10  rank          256 non-null    int64  
 11  imageUrl      250 non-null    object 
 12  imagesCount   250 non-null    float64
 13  reviewsCount  256 non-null    int64  
 14  scrapedAt     256 non-null    object 
 15  searchString  256 non-null    object 
dtypes: float64(4), int64(2), object(10)
memory usage: 32.1+ KB


In [23]:
# Exportamos a .CSV
df_juarez_01.to_csv('../data/processed/01_dataPrep/juarez/01_juarez.csv', index = False)

In [24]:
df_juarez_02 = pd.read_csv('../data/raw/juarez_dataset_crawler-google-places_2025-06-15_23-27-55-877.csv')
df_juarez_02.head(5)

,additionalInfo/Accesibilidad/0/Entrada accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/2/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Ambiente/0/A la moda,additionalInfo/Ambiente/0/Agradable,additionalInfo/Ambiente/1/Agradable,additionalInfo/Ambiente/1/Informal,additionalInfo/Ambiente/2/Informal,additionalInfo/Ambiente/2/Relajado,additionalInfo/Ambiente/3/Relajado,additionalInfo/Aspectos destacados/0/Asientos en la terraza,additionalInfo/Aspectos destacados/0/Buen café,additionalInfo/Aspectos destacados/0/Buenos productos frescos,additionalInfo/Aspectos destacados/1/Buen café,additionalInfo/Aspectos destacados/1/Deliciosos postres,additionalInfo/Aspectos destacados/2/Chimenea,additionalInfo/Aspectos destacados/3/Deliciosos postres,additionalInfo/Aspectos destacados/4/Música en vivo,additionalInfo/Aspectos destacados/5/Muy buena selección de cervezas,additionalInfo/Estacionamiento/0/Estacionamiento gratuito en la calle,additionalInfo/Información proporcionada por la empresa/0/Se identifica como mujer empresaria,additionalInfo/Mascotas/0/Se permiten perros,additionalInfo/Menores/0/Ideal para ir con niños,additionalInfo/Menores/1/Menú para niños,additionalInfo/Menores/1/Sillas altas,additionalInfo/Menores/2/Sillas altas,additionalInfo/Opciones de servicio/0/Asientos al aire libre,additionalInfo/Opciones de servicio/0/Compras en tienda,additionalInfo/Opciones de servicio/0/Entrega a domicilio,additionalInfo/Opciones de servicio/0/Entrega el mismo día,additionalInfo/Opciones de servicio/0/Retiros en la puerta,additionalInfo/Opciones de servicio/1/Compras en tienda,additionalInfo/Opciones de servicio/1/Entrega a domicilio,additionalInfo/Opciones de servicio/1/Entrega el mismo día,additionalInfo/Opciones de servicio/1/Entrega sin contacto,additionalInfo/Opciones de servicio/1/Para llevar,additionalInfo/Opciones de servicio/1/Retiro en tienda,additionalInfo/Opciones de servicio/1/Servicios en el lugar,additionalInfo/Opciones de servicio/2/Compras en tienda,additionalInfo/Opciones de servicio/2/Consumo en el lugar,additionalInfo/Opciones de servicio/2/Entrega a domicilio,additionalInfo/Opciones de servicio/2/Entrega el mismo día,additionalInfo/Opciones de servicio/2/Servicios en el lugar,additionalInfo/Opciones de servicio/3/Entrega a domicilio,additionalInfo/Opciones de servicio/3/Entrega el mismo día,additionalInfo/Opciones de servicio/3/Para llevar,additionalInfo/Opciones de servicio/4/Consumo en el lugar,additionalInfo/Opciones del local/0/Almuerzo,additionalInfo/Opciones del local/0/Desayunos,additionalInfo/Opciones del local/1/Almuerzo,additionalInfo/Opciones del local/1/Desayuno-almuerzo,additionalInfo/Opciones del local/2/Cena,additionalInfo/Opciones del local/2/Postres,additionalInfo/Opciones del local/3/Espacio con asientos,additionalInfo/Opciones del local/3/Servicio de catering,additionalInfo/Opciones del local/4/Postres,additionalInfo/Opciones del local/5/Espacio con asientos,additionalInfo/Pagos/0/Cheques,additionalInfo/Pagos/0/Pagos móviles mediante NFC,additionalInfo/Pagos/0/Tarjetas de crédito,additionalInfo/Pagos/0/Tarjetas de débito,additionalInfo/Pagos/1/Pagos móviles mediante NFC,additionalInfo/Pagos/1/Solo dinero en efectivo,additionalInfo/Pagos/1/Tarjetas de crédito,additionalInfo/Pagos/1/Tarjetas de débito,additionalInfo/Pagos/2/Tarjetas de crédito,additionalInfo/Pagos/2/Tarjetas de débito,additionalInfo/Pagos/3/Tarjetas de crédito,additionalInfo/Pagos/3/Tarjetas de débito,additionalInfo/Pagos/4/Tarjetas de crédito,additionalInfo/Planificaci

In [25]:
df_juarez_02.shape

(131, 527)

In [26]:
df_juarez_02 = df_juarez_02[[
        'title',
        'categoryName',
        'website',
        'street',
        'phone',
        'city',
        'state',
        'location/lat', 
        'location/lng',
        'totalScore',
        'rank',
        'imageUrl',
        'imagesCount',       
        'reviewsCount', 
        'scrapedAt',
        'searchString']]
df_juarez_02.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Agronomia y Vivero FANTINI,Ingeniero agrónomo,http://www.fantinihnos.com.ar/,"Ruta 86 y, Av. Constitución",+54 2281 40-8509,Benito Juárez,Provincia de Buenos Aires,-37.678079,-59.820755,5.0,13,https://lh3.googleusercontent.com/p/AF1QipOVmH...,3.0,2,2025-06-15T23:27:13.052Z,maquinaria
1,Lo de rosa,Kiosco,NaN,Palacio 164,NaN,Benito Juárez,Provincia de Buenos Aires,-37.675689,-59.794287,NaN,40,https://lh3.googleusercontent.com/p/AF1QipPaXl...,8.0,0,2025-06-15T23:27:17.780Z,tienda
2,Distribuidora MAGICA,Tienda de alimentación,NaN,NaN,NaN,Benito Juárez,Provincia de Buenos Aires,-37.677326,-59.804705,NaN,39,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,0,2025-06-15T23:27:17.780Z,tienda
3,Diaz Martin J,Carnicería,NaN,Chacabuco 290,+54 2292 45-3300,Benito Juárez,Provincia de Buenos Aires,-37.676439,-59.798500,4.0,60,https://lh3.googleusercontent.com/gps-cs-s/AC9...,16.0,1,2025-06-15T23:27:19.011Z,tienda
4,Elegante sport,Tienda de ropa de deportes,https://www.instagram.com/elegante_sport_bj?ig...,Ameghino 25,+54 2281 30-1465,Benito Juárez,Provincia de Buenos Aires,-37.681722,-59.815709,NaN,59,https://lh3.googleusercontent.com/p/AF1QipPPZN...,2.0,0,2025-06-15T23:27:19.011Z,tienda


In [27]:
df_juarez_02.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         131 non-null    object 
 1   categoryName  126 non-null    object 
 2   website       21 non-null     object 
 3   street        123 non-null    object 
 4   phone         80 non-null     object 
 5   city          131 non-null    object 
 6   state         131 non-null    object 
 7   location/lat  131 non-null    float64
 8   location/lng  131 non-null    float64
 9   totalScore    87 non-null     float64
 10  rank          131 non-null    int64  
 11  imageUrl      126 non-null    object 
 12  imagesCount   126 non-null    float64
 13  reviewsCount  131 non-null    int64  
 14  scrapedAt     131 non-null    object 
 15  searchString  131 non-null    object 
dtypes: float64(4), int64(2), object(10)
memory usage: 16.5+ KB


In [28]:
df_juarez_02.to_csv('../data/processed/01_dataPrep/juarez/02_juarez.csv',  index = False)

In [29]:
# Listar todos los CSV en la carpeta 'juarez' usando glob y patrón '*.csv'
glob.glob('../data/processed/01_dataPrep/juarez/*.csv')

['../data/processed/01_dataPrep/juarez/01_juarez.csv',
 '../data/processed/01_dataPrep/juarez/02_juarez.csv']

In [30]:
# Lee todos los CSV de la carpeta 'juarez' y los combina en un solo DataFrame:
# glob.glob encuentra los archivos .csv, pd.read_csv los lee uno por uno,
# map aplica la lectura sobre cada ruta, y pd.concat los une fila a fila.
df = pd.concat(map(pd.read_csv, glob.glob('../data/processed/01_dataPrep/juarez/*.csv')))
df.columns

Index(['title', 'categoryName', 'website', 'street', 'phone', 'city', 'state',
       'location/lat', 'location/lng', 'totalScore', 'rank', 'imageUrl',
       'imagesCount', 'reviewsCount', 'scrapedAt', 'searchString'],
      dtype='object')

In [31]:
# Exporta el DataFrame a .CSV sin incluir la columna del índice
df.to_csv('../data/processed/01_dataPrep/main/main_juarez.csv', index=False)

In [32]:
df_olavarria_01 = pd.read_csv('../data/raw/olavarria_dataset_crawler-google-places_2025-05-29_00-56-36-905.csv')
df_olavarria_01.head(5)

,additionalInfo/Accesibilidad/0/Entrada accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/2/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Ambiente/0/A la moda,additionalInfo/Ambiente/1/Agradable,additionalInfo/Ambiente/2/Informal,additionalInfo/Ambiente/3/Relajado,additionalInfo/Aspectos destacados/0/Buen café,additionalInfo/Aspectos destacados/0/Buenos productos frescos,additionalInfo/Aspectos destacados/0/Chimenea,additionalInfo/Aspectos destacados/0/Muy buena selección de cervezas,additionalInfo/Aspectos destacados/1/Deliciosos postres,additionalInfo/Estacionamiento/0/Estacionamiento gratuito en la calle,additionalInfo/Información proporcionada por la empresa/0/Se identifica como mujer empresaria,additionalInfo/Menores/0/Ideal para ir con niños,additionalInfo/Opciones de servicio/0/Citas en línea,additionalInfo/Opciones de servicio/0/Compras en tienda,additionalInfo/Opciones de servicio/0/Consumo en el lugar,additionalInfo/Opciones de servicio/0/Entrega a domicilio,additionalInfo/Opciones de servicio/0/Entrega el mismo día,additionalInfo/Opciones de servicio/0/Entrega sin contacto,additionalInfo/Opciones de servicio/0/Retiros en la puerta,additionalInfo/Opciones de servicio/0/Servicios en el lugar,additionalInfo/Opciones de servicio/1/Citas en línea,additionalInfo/Opciones de servicio/1/Compras en tienda,additionalInfo/Opciones de servicio/1/Entrega a domicilio,additionalInfo/Opciones de servicio/1/Entrega el mismo día,additionalInfo/Opciones de servicio/1/Retiro en tienda,additionalInfo/Opciones de servicio/1/Servicios en el lugar,additionalInfo/Opciones de servicio/2/Compras en tienda,additionalInfo/Opciones de servicio/2/Entrega el mismo día,additionalInfo/Opciones de servicio/2/Para llevar,additionalInfo/Opciones de servicio/2/Retiro en tienda,additionalInfo/Opciones de servicio/2/Servicios en el lugar,additionalInfo/Opciones de servicio/3/Compras en tienda,additionalInfo/Opciones de servicio/3/Consumo en el lugar,additionalInfo/Opciones de servicio/3/Entrega el mismo día,additionalInfo/Opciones de servicio/4/Entrega el mismo día,additionalInfo/Opciones de servicio/4/Servicios en el lugar,additionalInfo/Opciones de servicio/5/Entrega el mismo día,additionalInfo/Opciones de servicio/6/Citas en línea,additionalInfo/Opciones del local/0/Almuerzo,additionalInfo/Opciones del local/0/Postres,additionalInfo/Opciones del local/1/Cena,additionalInfo/Pagos/0/Cheques,additionalInfo/Pagos/0/Pagos móviles mediante NFC,additionalInfo/Pagos/0/Solo dinero en efectivo,additionalInfo/Pagos/0/Tarjetas de crédito,additionalInfo/Pagos/0/Tarjetas de débito,additionalInfo/Pagos/1/Pagos móviles mediante NFC,additionalInfo/Pagos/1/Solo dinero en efectivo,additionalInfo/Pagos/1/Tarjetas de crédito,additionalInfo/Pagos/1/Tarjetas de débito,additionalInfo/Pagos/2/Tarjetas de crédito,additionalInfo/Pagos/2/Tarjetas de débito,additionalInfo/Pagos/3/Tarjetas de crédito,additionalInfo/Pagos/3/Tarjetas de débito,additionalInfo/Pagos/4/Tarjetas de crédito,additionalInfo/Planificación/0/Se aceptan reservas,additionalInfo/Planificación/0/Se recomienda concertar cita,additionalInfo/Planificación/0/Se requiere una cita,additionalInfo/Planificación/0/Visita rápida,additionalInfo/Planificación/1/Visita rápida,additionalInfo/Popular por/0/Almuerzo,additionalInfo/Popular por/1/Cena,additionalInfo/Público usual/0/Amigable con LGBTQ+,additionalInfo/Público usual/0/Recomendable para ir en familia,additionalInfo/Qué ofrece/0/Alcohol,additionalInfo/Qué ofrece/0/Bocadillos,additionalInfo/Qué ofrece/0/Productos orgánicos,additionalInfo/Qué ofrece/0/Servicio de armado,additionalInfo/Qué ofrece/1/Café,additionalInfo/Qué ofrece/1/Cerveza,additionalInfo/Qué ofrece/1/Servicios de reparación,additionalInfo/Qué ofrece/2/Comida,additionalInfo/Qué ofrece/

In [33]:
df_olavarria_01.shape

(206, 145)

In [34]:
df_olavarria_01 = df_olavarria_01[[
        'title',
        'categoryName',
        'website',
        'street',
        'phone',
        'city',
        'state',
        'location/lat', 
        'location/lng',
        'totalScore',
        'rank',
        'imageUrl',
        'imagesCount',       
        'reviewsCount', 
        'scrapedAt',
        'searchString']]
df_olavarria_01.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Polirrubros La Silvia,Kiosco,NaN,Av. Ituzaingó 1317,NaN,Olavarría,Provincia de Buenos Aires,-36.895466,-60.345100,4.8,1,https://lh3.googleusercontent.com/gps-cs-s/AC9...,16.0,27,2025-05-29T00:56:12.806Z,comercio
1,Distribuidora Katy,Comercio,NaN,"Pelegrino, Av. Ituzaingó e",+54 2284 71-1461,Olavarría,Provincia de Buenos Aires,-36.890448,-60.338133,NaN,2,https://lh3.googleusercontent.com/p/AF1QipOckQ...,20.0,0,2025-05-29T00:56:12.808Z,comercio
2,Mercadito Gaby-Fer,Comercio,NaN,Collinet 2595,+54 2284 41-7540,Olavarría,Provincia de Buenos Aires,-36.900998,-60.335121,5.0,3,https://lh3.googleusercontent.com/gps-cs-s/AC9...,3.0,4,2025-05-29T00:56:12.808Z,comercio
3,Dulcemente,Tienda de alimentación,NaN,Independencia 1700-1600,+54 2284 69-7125,Olavarría,Provincia de Buenos Aires,-36.895493,-60.339700,5.0,4,https://lh3.googleusercontent.com/gps-cs-s/AC9...,2.0,2,2025-05-29T00:56:12.808Z,comercio
4,104 Polirrubro,Tienda de alimentación,NaN,Azopardo 1598,+54 2284 42-1670,Olavarría,Provincia de Buenos Aires,-36.893022,-60.341844,3.8,5,https://lh3.googleusercontent.com/gps-cs-s/AC9...,18.0,4,2025-05-29T00:56:12.808Z,comercio


In [35]:
df_olavarria_01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         206 non-null    object 
 1   categoryName  201 non-null    object 
 2   website       48 non-null     object 
 3   street        195 non-null    object 
 4   phone         151 non-null    object 
 5   city          205 non-null    object 
 6   state         205 non-null    object 
 7   location/lat  206 non-null    float64
 8   location/lng  206 non-null    float64
 9   totalScore    163 non-null    float64
 10  rank          206 non-null    int64  
 11  imageUrl      200 non-null    object 
 12  imagesCount   200 non-null    float64
 13  reviewsCount  206 non-null    int64  
 14  scrapedAt     206 non-null    object 
 15  searchString  206 non-null    object 
dtypes: float64(4), int64(2), object(10)
memory usage: 25.9+ KB


In [36]:
df_olavarria_01.to_csv('../data/processed/01_dataPrep/olavarria/01_olavarria.csv',  index = False)

In [37]:
df_olavarria_02 = pd.read_csv('../data/raw/olavarria_dataset_crawler-google-places_2025-06-11_18-14-26-674.csv')
df_olavarria_02.head(5)

,additionalInfo/Accesibilidad/0/Entrada accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/2/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/2/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/3/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Ambiente/0/A la moda,additionalInfo/Ambiente/0/Informal,additionalInfo/Ambiente/1/Agradable,additionalInfo/Ambiente/1/Relajado,additionalInfo/Ambiente/2/Informal,additionalInfo/Ambiente/3/Relajado,additionalInfo/Aspectos destacados/0/Buen café,additionalInfo/Aspectos destacados/0/Buenos productos frescos,additionalInfo/Aspectos destacados/0/Chimenea,additionalInfo/Aspectos destacados/0/Muy buena selección de cervezas,additionalInfo/Aspectos destacados/0/Muy buena selección de vinos,additionalInfo/Aspectos destacados/1/Deliciosos postres,additionalInfo/Estacionamiento/0/Estacionamiento en el lugar,additionalInfo/Estacionamiento/0/Estacionamiento gratuito,additionalInfo/Estacionamiento/0/Estacionamiento gratuito en la calle,additionalInfo/Estacionamiento/1/Estacionamiento gratuito,additionalInfo/Estacionamiento/1/Estacionamiento gratuito en la calle,additionalInfo/Estacionamiento/2/Estacionamiento gratuito en la calle,additionalInfo/Información proporcionada por la empresa/0/Se identifica como mujer empresaria,additionalInfo/Menores/0/Ideal para ir con niños,additionalInfo/Opciones de servicio/0/Citas en línea,additionalInfo/Opciones de servicio/0/Compras en tienda,additionalInfo/Opciones de servicio/0/Consumo en el lugar,additionalInfo/Opciones de servicio/0/Entrega a domicilio,additionalInfo/Opciones de servicio/0/Entrega el mismo día,additionalInfo/Opciones de servicio/0/Entrega sin contacto,additionalInfo/Opciones de servicio/0/Para llevar,additionalInfo/Opciones de servicio/0/Retiro en tienda,additionalInfo/Opciones de servicio/0/Retiros en la puerta,additionalInfo/Opciones de servicio/0/Servicios en el lugar,additionalInfo/Opciones de servicio/1/Citas en línea,additionalInfo/Opciones de servicio/1/Compras en tienda,additionalInfo/Opciones de servicio/1/Consumo en el lugar,additionalInfo/Opciones de servicio/1/Entrega a domicilio,additionalInfo/Opciones de servicio/1/Entrega el mismo día,additionalInfo/Opciones de servicio/1/Entrega sin contacto,additionalInfo/Opciones de servicio/1/Para llevar,additionalInfo/Opciones de servicio/1/Retiro en tienda,additionalInfo/Opciones de servicio/1/Servicios en el lugar,additionalInfo/Opciones de servicio/2/Compras en tienda,additionalInfo/Opciones de servicio/2/Consumo en el lugar,additionalInfo/Opciones de servicio/2/Entrega a domicilio,additionalInfo/Opciones de servicio/2/Entrega el mismo día,additionalInfo/Opciones de servicio/2/Para llevar,additionalInfo/Opciones de servicio/2/Retiro en tienda,additionalInfo/Opciones de servicio/3/Compras en tienda,additionalInfo/Opciones de servicio/3/Consumo en el lugar,additionalInfo/Opciones de servicio/3/Entrega el mismo día,additionalInfo/Opciones de servicio/3/Para llevar,additionalInfo/Opciones de servicio/3/Retiro en tienda,additionalInfo/Opciones de servicio/4/Entrega el mismo día,additionalInfo/Opciones de servicio/4/Servicios en el lugar,additionalInfo/Opciones de servicio/5/Entrega el mismo día,additionalInfo/Opciones de servicio/6/Citas en línea,additionalInfo/Opciones del local/0/Almuerzo,additionalInfo/Opciones del local/0/Postres,additionalInfo/Opciones del local/1/Cena,a

In [38]:
df_olavarria_02.shape

(511, 261)

In [39]:
df_olavarria_02 = df_olavarria_02[[
        'title',
        'categoryName',
        'website',
        'street',
        'phone',
        'city',
        'state',
        'location/lat', 
        'location/lng',
        'totalScore',
        'rank',
        'imageUrl',
        'imagesCount',       
        'reviewsCount', 
        'scrapedAt',
        'searchString']]
df_olavarria_02.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Vazzano Remates,Tienda de muebles,https://www.facebook.com/rematesvazzano,Av. Urquiza 1850,+54 2284 44-4646,Olavarría,Provincia de Buenos Aires,-36.908371,-60.320210,4.4,1,https://lh3.googleusercontent.com/p/AF1QipO6OX...,42.0,145,2025-06-11T18:13:55.853Z,venta online
1,Sweet Ness S.a.,Comercio,NaN,Av. Urquiza 1746,NaN,Olavarría,Provincia de Buenos Aires,-36.908706,-60.321266,4.6,2,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,10,2025-06-11T18:13:55.855Z,venta online
2,Telemundo,Tienda de móviles,NaN,Necochea 2893,+54 2284 41-7777,Olavarría,Provincia de Buenos Aires,-36.892333,-60.319047,4.5,3,https://lh3.googleusercontent.com/gps-cs-s/AC9...,3.0,20,2025-06-11T18:13:55.855Z,venta online
3,Tienda Online Olavarria,NaN,https://sites.google.com/view/tiendaonlineolav...,Antártida Argentina 606,+54 2284 68-2953,Olavarría,Provincia de Buenos Aires,-36.921672,-60.325226,NaN,4,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,0,2025-06-11T18:13:55.855Z,venta online
4,AS Estufas Olavarría,Tienda de estufas de leña,NaN,Av. Avellaneda 3043,+54 2284 41-1312,IDH,Provincia de Buenos Aires,-36.907327,-60.335393,4.6,5,https://lh3.googleusercontent.com/gps-proxy/AL...,20.0,14,2025-06-11T18:13:55.855Z,venta online


In [40]:
df_olavarria_02.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511 entries, 0 to 510
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         511 non-null    object 
 1   categoryName  507 non-null    object 
 2   website       163 non-null    object 
 3   street        497 non-null    object 
 4   phone         385 non-null    object 
 5   city          510 non-null    object 
 6   state         510 non-null    object 
 7   location/lat  511 non-null    float64
 8   location/lng  511 non-null    float64
 9   totalScore    406 non-null    float64
 10  rank          511 non-null    int64  
 11  imageUrl      507 non-null    object 
 12  imagesCount   507 non-null    float64
 13  reviewsCount  511 non-null    int64  
 14  scrapedAt     511 non-null    object 
 15  searchString  511 non-null    object 
dtypes: float64(4), int64(2), object(10)
memory usage: 64.0+ KB


In [41]:
df_olavarria_02.to_csv('../data/processed/01_dataPrep/olavarria/02_olavarria.csv',  index = False)

In [42]:
df_olavarria_03 = pd.read_csv('../data/raw/olavarria_dataset_crawler-google-places_2025-06-15_23-35-08-815.csv')
df_olavarria_03.head(5)

,additionalInfo/Accesibilidad/0/Bucle magnético de asistencia,additionalInfo/Accesibilidad/0/Entrada accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/2/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Ambiente/0/A la moda,additionalInfo/Ambiente/1/Agradable,additionalInfo/Ambiente/2/Informal,additionalInfo/Ambiente/3/Relajado,additionalInfo/Aspectos destacados/0/Buenos productos frescos,additionalInfo/Aspectos destacados/0/Chimenea,additionalInfo/Aspectos destacados/0/Muy buena selección de cervezas,additionalInfo/Estacionamiento/0/Estacionamiento en el lugar,additionalInfo/Estacionamiento/0/Estacionamiento gratuito,additionalInfo/Estacionamiento/0/Estacionamiento gratuito en la calle,additionalInfo/Estacionamiento/1/Estacionamiento gratuito,additionalInfo/Estacionamiento/1/Estacionamiento gratuito en la calle,additionalInfo/Estacionamiento/2/Estacionamiento gratuito en la calle,additionalInfo/Estacionamiento/3/Garage de stacionamiento gratuito,additionalInfo/Información proporcionada por la empresa/0/Se identifica como mujer empresaria,additionalInfo/Menores/0/Ideal para ir con niños,additionalInfo/Opciones de servicio/0/Citas en línea,additionalInfo/Opciones de servicio/0/Compras en tienda,additionalInfo/Opciones de servicio/0/Consumo en el lugar,additionalInfo/Opciones de servicio/0/Entrega a domicilio,additionalInfo/Opciones de servicio/0/Entrega el mismo día,additionalInfo/Opciones de servicio/0/Entrega sin contacto,additionalInfo/Opciones de servicio/0/Retiro en tienda,additionalInfo/Opciones de servicio/0/Retiros en la puerta,additionalInfo/Opciones de servicio/0/Servicios en el lugar,additionalInfo/Opciones de servicio/1/Citas en línea,additionalInfo/Opciones de servicio/1/Compras en tienda,additionalInfo/Opciones de servicio/1/Entrega a domicilio,additionalInfo/Opciones de servicio/1/Entrega el mismo día,additionalInfo/Opciones de servicio/1/Entrega sin contacto,additionalInfo/Opciones de servicio/1/Retiro en tienda,additionalInfo/Opciones de servicio/1/Servicios en el lugar,additionalInfo/Opciones de servicio/2/Compras en tienda,additionalInfo/Opciones de servicio/2/Entrega a domicilio,additionalInfo/Opciones de servicio/2/Entrega el mismo día,additionalInfo/Opciones de servicio/2/Para llevar,additionalInfo/Opciones de servicio/2/Retiro en tienda,additionalInfo/Opciones de servicio/2/Servicios en el lugar,additionalInfo/Opciones de servicio/3/Compras en tienda,additionalInfo/Opciones de servicio/3/Consumo en el lugar,additionalInfo/Opciones de servicio/3/Entrega el mismo día,additionalInfo/Opciones de servicio/3/Retiro en tienda,additionalInfo/Opciones de servicio/4/Entrega el mismo día,additionalInfo/Opciones de servicio/4/Servicios en el lugar,additionalInfo/Opciones de servicio/5/Entrega el mismo día,additionalInfo/Opciones de servicio/6/Citas en línea,additionalInfo/Opciones del local/0/Almuerzo,additionalInfo/Opciones del local/1/Cena,additionalInfo/Pagos/0/Cheques,additionalInfo/Pagos/0/Pagos móviles mediante NFC,additionalInfo/Pagos/0/Solo dinero en efectivo,additionalInfo/Pagos/0/Tarjetas de crédito,additionalInfo/Pagos/0/Tarjetas de débito,additionalInfo/Pagos/1/Pagos móviles mediante NFC,additionalInfo/Pagos/1/Solo dinero en efectivo,additionalInfo/Pagos/1/Tarjetas de crédito,additionalInfo/Pagos/1/Tarjetas de débito,additionalInfo/Pagos/2/Tarjetas de crédito,additionalInfo/Pagos/2/Tarjetas de débito,additionalInfo/Pagos/3/Tarjetas de crédito,additionalInfo/

In [43]:
df_olavarria_03.shape

(376, 261)

In [44]:
df_olavarria_03 = df_olavarria_03[[
        'title',
        'categoryName',
        'website',
        'street',
        'phone',
        'city',
        'state',
        'location/lat', 
        'location/lng',
        'totalScore',
        'rank',
        'imageUrl',
        'imagesCount',       
        'reviewsCount', 
        'scrapedAt',
        'searchString']]
df_olavarria_03.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Alto Salame Olavarria,Tienda de alimentación,NaN,Av. Domingo Faustino Sarmiento 2130,+54 2284 62-2206,Olavarría,Provincia de Buenos Aires,-36.881409,-60.317939,NaN,20,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,0,2025-06-15T23:31:30.165Z,comercio
1,"Frutería y Verdulería ""El Nuevo Galpón""",Frutería,NaN,Av. de los Trabajadores 4312,+54 9 2284 35-5136,Olavarría,Provincia de Buenos Aires,-36.892194,-60.294718,4.5,19,https://lh3.googleusercontent.com/gps-cs-s/AC9...,18.0,15,2025-06-15T23:31:30.165Z,comercio
2,Despensa Bauti,Tienda de alimentación,NaN,Cerrito 4300,+54 9 2284 23-8406,Olavarría,Provincia de Buenos Aires,-36.879480,-60.314791,NaN,18,https://lh3.googleusercontent.com/gps-proxy/AL...,4.0,0,2025-06-15T23:31:30.165Z,comercio
3,Toxica,Tienda de ropa de mujer,NaN,Necochea 2757,NaN,Olavarría,Provincia de Buenos Aires,-36.891471,-60.320165,5.0,66,https://lh3.googleusercontent.com/gps-cs-s/AC9...,2.0,1,2025-06-15T23:31:33.614Z,comercio
4,"Kiosco ""El Quijote""",Quiosco,NaN,Lavalle 2880,NaN,Olavarría,Provincia de Buenos Aires,-36.891331,-60.324035,5.0,65,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,2,2025-06-15T23:31:33.614Z,comercio


In [45]:
df_olavarria_03.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376 entries, 0 to 375
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         376 non-null    object 
 1   categoryName  361 non-null    object 
 2   website       138 non-null    object 
 3   street        364 non-null    object 
 4   phone         297 non-null    object 
 5   city          375 non-null    object 
 6   state         375 non-null    object 
 7   location/lat  376 non-null    float64
 8   location/lng  376 non-null    float64
 9   totalScore    306 non-null    float64
 10  rank          376 non-null    int64  
 11  imageUrl      375 non-null    object 
 12  imagesCount   375 non-null    float64
 13  reviewsCount  376 non-null    int64  
 14  scrapedAt     376 non-null    object 
 15  searchString  376 non-null    object 
dtypes: float64(4), int64(2), object(10)
memory usage: 47.1+ KB


In [46]:
df_olavarria_03.to_csv('../data/processed/01_dataPrep/olavarria/03_olavarria.csv',  index = False)

In [47]:
glob.glob('../data/processed/01_dataPrep/olavarria/*.csv')

['../data/processed/01_dataPrep/olavarria/01_olavarria.csv',
 '../data/processed/01_dataPrep/olavarria/03_olavarria.csv',
 '../data/processed/01_dataPrep/olavarria/02_olavarria.csv']

In [48]:
df = pd.concat(map(pd.read_csv, glob.glob('../data/processed/01_dataPrep/olavarria/*.csv')))
df.columns

Index(['title', 'categoryName', 'website', 'street', 'phone', 'city', 'state',
       'location/lat', 'location/lng', 'totalScore', 'rank', 'imageUrl',
       'imagesCount', 'reviewsCount', 'scrapedAt', 'searchString'],
      dtype='object')

In [49]:
df.to_csv('../data/processed/01_dataPrep/main/main_olavarria.csv', index=False)

In [50]:
df = pd.read_csv('../data/processed/01_dataPrep/main/main_olavarria.csv')
df.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Polirrubros La Silvia,Kiosco,NaN,Av. Ituzaingó 1317,NaN,Olavarría,Provincia de Buenos Aires,-36.895466,-60.345100,4.8,1,https://lh3.googleusercontent.com/gps-cs-s/AC9...,16.0,27,2025-05-29T00:56:12.806Z,comercio
1,Distribuidora Katy,Comercio,NaN,"Pelegrino, Av. Ituzaingó e",+54 2284 71-1461,Olavarría,Provincia de Buenos Aires,-36.890448,-60.338133,NaN,2,https://lh3.googleusercontent.com/p/AF1QipOckQ...,20.0,0,2025-05-29T00:56:12.808Z,comercio
2,Mercadito Gaby-Fer,Comercio,NaN,Collinet 2595,+54 2284 41-7540,Olavarría,Provincia de Buenos Aires,-36.900998,-60.335121,5.0,3,https://lh3.googleusercontent.com/gps-cs-s/AC9...,3.0,4,2025-05-29T00:56:12.808Z,comercio
3,Dulcemente,Tienda de alimentación,NaN,Independencia 1700-1600,+54 2284 69-7125,Olavarría,Provincia de Buenos Aires,-36.895493,-60.339700,5.0,4,https://lh3.googleusercontent.com/gps-cs-s/AC9...,2.0,2,2025-05-29T00:56:12.808Z,comercio
4,104 Polirrubro,Tienda de alimentación,NaN,Azopardo 1598,+54 2284 42-1670,Olavarría,Provincia de Buenos Aires,-36.893022,-60.341844,3.8,5,https://lh3.googleusercontent.com/gps-cs-s/AC9...,18.0,4,2025-05-29T00:56:12.808Z,comercio


In [51]:
df_rauch_01= pd.read_csv('../data/raw/rauch_ dataset_crawler-google-places_2025-05-29_00-49-55-809.csv')
df_rauch_01.head(5)

,additionalInfo/Accesibilidad/0/Entrada accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/2/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Ambiente/0/A la moda,additionalInfo/Ambiente/1/Agradable,additionalInfo/Ambiente/2/Informal,additionalInfo/Ambiente/3/Relajado,additionalInfo/Aspectos destacados/0/Buenos productos frescos,additionalInfo/Estacionamiento/0/Estacionamiento gratuito,additionalInfo/Estacionamiento/0/Estacionamiento gratuito en la calle,additionalInfo/Estacionamiento/0/Estacionamiento pagado,additionalInfo/Estacionamiento/1/Estacionamiento gratuito en la calle,additionalInfo/Estacionamiento/2/Garage de stacionamiento gratuito,additionalInfo/Información proporcionada por la empresa/0/Se identifica como mujer empresaria,additionalInfo/Mascotas/0/Se permiten perros,additionalInfo/Menores/0/Ideal para ir con niños,additionalInfo/Opciones de servicio/0/Entrega a domicilio,additionalInfo/Opciones de servicio/0/Entrega el mismo día,additionalInfo/Opciones de servicio/0/Entrega sin contacto,additionalInfo/Opciones de servicio/0/Retiro en tienda,additionalInfo/Opciones de servicio/0/Retiros en la puerta,additionalInfo/Opciones de servicio/1/Compras en tienda,additionalInfo/Opciones de servicio/1/Entrega a domicilio,additionalInfo/Opciones de servicio/1/Entrega el mismo día,additionalInfo/Opciones de servicio/1/Para llevar,additionalInfo/Opciones de servicio/1/Retiro en tienda,additionalInfo/Opciones de servicio/2/Compras en tienda,additionalInfo/Opciones de servicio/2/Entrega el mismo día,additionalInfo/Opciones de servicio/2/Para llevar,additionalInfo/Opciones de servicio/2/Retiro en tienda,additionalInfo/Opciones de servicio/2/Servicios en el lugar,additionalInfo/Opciones de servicio/3/Compras en tienda,additionalInfo/Opciones de servicio/3/Consumo en el lugar,additionalInfo/Opciones de servicio/3/Entrega a domicilio,additionalInfo/Opciones de servicio/3/Entrega el mismo día,additionalInfo/Opciones de servicio/3/Para llevar,additionalInfo/Opciones de servicio/4/Consumo en el lugar,additionalInfo/Opciones de servicio/4/Entrega el mismo día,additionalInfo/Opciones de servicio/5/Entrega el mismo día,additionalInfo/Pagos/0/Cheques,additionalInfo/Pagos/0/Pagos móviles mediante NFC,additionalInfo/Pagos/0/Solo dinero en efectivo,additionalInfo/Pagos/0/Tarjetas de crédito,additionalInfo/Pagos/0/Tarjetas de débito,additionalInfo/Pagos/1/Pagos móviles mediante NFC,additionalInfo/Pagos/1/Solo dinero en efectivo,additionalInfo/Pagos/1/Tarjetas de crédito,additionalInfo/Pagos/1/Tarjetas de débito,additionalInfo/Pagos/2/Tarjetas de crédito,additionalInfo/Pagos/2/Tarjetas de débito,additionalInfo/Pagos/3/Tarjetas de crédito,additionalInfo/Pagos/3/Tarjetas de débito,additionalInfo/Pagos/4/Tarjetas de crédito,additionalInfo/Planificación/0/Se requiere una cita,additionalInfo/Planificación/0/Visita rápida,additionalInfo/Público usual/0/Amigable con LGBTQ+,additionalInfo/Público usual/0/Grupos,additionalInfo/Público usual/1/Espacio seguro para personas transgénero,additionalInfo/Qué ofrece/0/Café,additionalInfo/Qué ofrece/0/Combustible diésel,additionalInfo/Qué ofrece/0/Comida,additionalInfo/Qué ofrece/0/Comida preparada,additionalInfo/Qué ofrece/0/Servicio de armado,additionalInfo/Qué ofrece/1/Gasolinera con servicio completo,additionalInfo/Qué ofrece/1/Intercambio de tanques de propano,additionalInfo/Qué ofrece/2/Intercambio de tanques de propano,additionalInfo/Qué ofrece/2/Lavader

In [52]:
df_rauch_01.shape

(215, 186)

In [53]:
df_rauch_01 = df_rauch_01[[
        'title',
        'categoryName',
        'website',
        'street',
        'phone',
        'city',
        'state',
        'location/lat', 
        'location/lng',
        'totalScore',
        'rank',
        'imageUrl',
        'imagesCount',       
        'reviewsCount', 
        'scrapedAt',
        'searchString']]
df_rauch_01.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Del Valle inyeccion electronica,Taller de reparación de automóviles,NaN,"Aristóbulo del Valle, Aveleyra &",+54 249 434-0861,Rauch,Provincia de Buenos Aires,-36.773298,-59.091459,5.0,1,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,1,2025-05-29T00:48:32.392Z,electronica
1,Electro 440 - Accesorios y Reparación,Tienda de electrónica,https://www.facebook.com/todoenremotos,Letamendi 596,+54 9 249 427-7096,Rauch,Provincia de Buenos Aires,-36.776094,-59.088072,5.0,2,https://lh3.googleusercontent.com/gps-cs-s/AC9...,35.0,5,2025-05-29T00:48:32.394Z,electronica
2,Dangerous Sonido e Iluminación,Tienda de equipos de sonido,NaN,"BNK, Saavedra 507",+54 249 453-3029,Rauch,Provincia de Buenos Aires,-36.772975,-59.093649,5.0,3,https://lh3.googleusercontent.com/gps-cs-s/AC9...,28.0,3,2025-05-29T00:48:32.394Z,electronica
3,Jona PC,Servicio de informática,https://www.facebook.com/profile.php?id=100069...,Dardo Rocha 850,NaN,Rauch,Provincia de Buenos Aires,-36.761649,-59.092823,5.0,4,https://lh3.googleusercontent.com/p/AF1QipPcWb...,13.0,11,2025-05-29T00:48:32.394Z,electronica
4,Super PC Rauch,Tienda de informática,NaN,Balcarce 1001,+54 249 435-8292,Rauch,Provincia de Buenos Aires,-36.768249,-59.090148,NaN,5,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,0,2025-05-29T00:48:32.395Z,electronica


In [54]:
df_rauch_01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         215 non-null    object 
 1   categoryName  211 non-null    object 
 2   website       42 non-null     object 
 3   street        198 non-null    object 
 4   phone         120 non-null    object 
 5   city          214 non-null    object 
 6   state         214 non-null    object 
 7   location/lat  215 non-null    float64
 8   location/lng  215 non-null    float64
 9   totalScore    136 non-null    float64
 10  rank          215 non-null    int64  
 11  imageUrl      209 non-null    object 
 12  imagesCount   209 non-null    float64
 13  reviewsCount  215 non-null    int64  
 14  scrapedAt     215 non-null    object 
 15  searchString  215 non-null    object 
dtypes: float64(4), int64(2), object(10)
memory usage: 27.0+ KB


In [55]:
df_rauch_01.to_csv('../data/processed/01_dataPrep/rauch/01_rauch.csv',  index = False)

In [56]:
df_rauch_02 = pd.read_csv('../data/raw/rauch_dataset_crawler-google-places_2025-06-15_22-41-11-472.csv')
df_rauch_02.head(5)

,additionalInfo/Accesibilidad/0/Entrada accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Aspectos destacados/0/Buenos productos frescos,additionalInfo/Información proporcionada por la empresa/0/Se identifica como mujer empresaria,additionalInfo/Opciones de servicio/0/Entrega a domicilio,additionalInfo/Opciones de servicio/0/Entrega el mismo día,additionalInfo/Opciones de servicio/0/Entrega sin contacto,additionalInfo/Opciones de servicio/0/Retiro en tienda,additionalInfo/Opciones de servicio/1/Compras en tienda,additionalInfo/Opciones de servicio/1/Entrega a domicilio,additionalInfo/Opciones de servicio/1/Entrega el mismo día,additionalInfo/Opciones de servicio/1/Retiro en tienda,additionalInfo/Opciones de servicio/2/Compras en tienda,additionalInfo/Opciones de servicio/2/Entrega el mismo día,additionalInfo/Opciones de servicio/2/Retiro en tienda,additionalInfo/Opciones de servicio/3/Compras en tienda,additionalInfo/Opciones de servicio/3/Entrega el mismo día,additionalInfo/Opciones de servicio/4/Entrega el mismo día,additionalInfo/Pagos/0/Cheques,additionalInfo/Pagos/0/Pagos móviles mediante NFC,additionalInfo/Pagos/0/Tarjetas de crédito,additionalInfo/Pagos/0/Tarjetas de débito,additionalInfo/Pagos/1/Pagos móviles mediante NFC,additionalInfo/Pagos/1/Tarjetas de crédito,additionalInfo/Pagos/1/Tarjetas de débito,additionalInfo/Pagos/2/Tarjetas de crédito,additionalInfo/Pagos/2/Tarjetas de débito,additionalInfo/Pagos/3/Tarjetas de crédito,additionalInfo/Pagos/3/Tarjetas de débito,additionalInfo/Planificación/0/Visita rápida,additionalInfo/Público usual/0/Amigable con LGBTQ+,additionalInfo/Público usual/1/Espacio seguro para personas transgénero,additionalInfo/Qué ofrece/0/Combustible diésel,additionalInfo/Qué ofrece/0/Comida,additionalInfo/Qué ofrece/0/Comida preparada,additionalInfo/Qué ofrece/1/Gasolinera con servicio completo,additionalInfo/Qué ofrece/2/Intercambio de tanques de propano,additionalInfo/Qué ofrece/3/Lavadero de autos,additionalInfo/Servicios/0/Minimercado,additionalInfo/Servicios/1/Sanitario,address,categories/0,categories/1,categories/2,categories/3,categoryName,cid,city,claimThisBusiness,countryCode,description,domain,emails/0,emails/1,facebooks/0,fid,googleFoodUrl,imageUrl,imagesCount,instagrams/0,instagrams/1,isAdvertisement,kgmid,language,linkedIns/0,location/lat,location/lng,neighborhood,openingHours/0/day,openingHours/0/hours,openingHours/1/day,openingHours/1/hours,openingHours/2/day,openingHours/2/hours,openingHours/3/day,openingHours/3/hours,openingHours/4/day,openingHours/4/hours,openingHours/5/day,openingHours/5/hours,openingHours/6/day,openingHours/6/hours,permanentlyClosed,phone,phoneUnformatted,phones/0,phones/1,phones/2,phonesUncertain/0,phonesUncertain/1,phonesUncertain/2,phonesUncertain/3,placeId,postalCode,price,rank,reviews,reviewsCount,reviewsDistribution/fiveStar,reviewsDistribution/fourStar,reviewsDistribution/oneStar,reviewsDistribution/threeStar,reviewsDistribution/twoStar,scrapedAt,searchPageUrl,searchString,state,street,temporarilyClosed,title,totalScore,twitters/0,url,website,youtubes/0
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"AMT, Almte. Brown 924, B7203 Rauch, Provincia ...",Proveedor de sistemas de seguridad,NaN,NaN,NaN,Proveedor de sistemas de seguridad,9394717965379040521,Rauch,True,AR,NaN,NaN,NaN,NaN,NaN,0x9597276b16100b4b:0x8260be36d365f509,NaN,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,NaN,NaN,False,/g/11bbrl4945,es-419,NaN,-36.775456,-59.082643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,+54 2297 44-2299,5.422974e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [57]:
df_rauch_02.shape

(136, 117)

In [58]:
df_rauch_02 = df_rauch_02[[
        'title',
        'categoryName',
        'website',
        'street',
        'phone',
        'city',
        'state',
        'location/lat', 
        'location/lng',
        'totalScore',
        'rank',
        'imageUrl',
        'imagesCount',       
        'reviewsCount', 
        'scrapedAt',
        'searchString']]
df_rauch_02.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Lippo Anibal Jose - Alarmas Electricidad - Tel...,Proveedor de sistemas de seguridad,NaN,"AMT, Almte. Brown 924",+54 2297 44-2299,Rauch,Provincia de Buenos Aires,-36.775456,-59.082643,4.2,1,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,9,2025-06-15T22:40:08.097Z,telefonia
1,"Impacto Digital, Soluciones Informáticas y Dig...",NaN,NaN,Av. San Martín 419,+54 249 421-8587,Rauch,Provincia de Buenos Aires,-36.776353,-59.090975,4.8,10,https://lh3.googleusercontent.com/gps-cs-s/AC9...,204.0,5,2025-06-15T22:40:10.238Z,muebles
2,La Casa del Colchón,Colchonería,NaN,NaN,NaN,Rauch,Provincia de Buenos Aires,-36.780455,-59.084500,NaN,8,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,0,2025-06-15T22:40:10.238Z,muebles
3,Carpintería NAM SH,Fábrica de muebles,http://www.carpinterianamsh.com/,"Eduardo Rodriguez S/N Esquina, Av. San Martín",+54 249 435-9919,Rauch,Provincia de Buenos Aires,-36.780340,-59.095569,NaN,7,https://lh3.googleusercontent.com/p/AF1QipO653...,11.0,0,2025-06-15T22:40:10.238Z,muebles
4,ZZELLA HOGAR,Tienda de muebles,NaN,Av. San Martín 555,NaN,Rauch,Provincia de Buenos Aires,-36.775434,-59.089857,NaN,6,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,0,2025-06-15T22:40:10.238Z,muebles


In [59]:
df_rauch_02.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         136 non-null    object 
 1   categoryName  132 non-null    object 
 2   website       22 non-null     object 
 3   street        122 non-null    object 
 4   phone         70 non-null     object 
 5   city          134 non-null    object 
 6   state         134 non-null    object 
 7   location/lat  136 non-null    float64
 8   location/lng  136 non-null    float64
 9   totalScore    85 non-null     float64
 10  rank          136 non-null    int64  
 11  imageUrl      132 non-null    object 
 12  imagesCount   132 non-null    float64
 13  reviewsCount  136 non-null    int64  
 14  scrapedAt     136 non-null    object 
 15  searchString  136 non-null    object 
dtypes: float64(4), int64(2), object(10)
memory usage: 17.1+ KB


In [60]:
df_rauch_02.to_csv('../data/processed/01_dataPrep/rauch/02_rauch.csv',  index = False)

In [61]:
df_rauch_03 = pd.read_csv('../data/raw/rauch_dataset_google-maps-extractor_2025-05-19_00-06-21-606.csv')
df_rauch_03.head(5)

,additionalInfo/Accesibilidad/0/Entrada accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/2/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Aspectos destacados/0/Buenos productos frescos,additionalInfo/Estacionamiento/0/Estacionamiento gratuito,additionalInfo/Estacionamiento/0/Estacionamiento pagado,additionalInfo/Estacionamiento/1/Estacionamiento gratuito en la calle,additionalInfo/Estacionamiento/2/Garage de stacionamiento gratuito,additionalInfo/Información proporcionada por la empresa/0/Se identifica como mujer empresaria,additionalInfo/Opciones de servicio/0/Entrega a domicilio,additionalInfo/Opciones de servicio/0/Entrega el mismo día,additionalInfo/Opciones de servicio/0/Entrega sin contacto,additionalInfo/Opciones de servicio/0/Retiro en tienda,additionalInfo/Opciones de servicio/0/Retiros en la puerta,additionalInfo/Opciones de servicio/1/Compras en tienda,additionalInfo/Opciones de servicio/1/Entrega a domicilio,additionalInfo/Opciones de servicio/1/Entrega el mismo día,additionalInfo/Opciones de servicio/1/Para llevar,additionalInfo/Opciones de servicio/1/Retiro en tienda,additionalInfo/Opciones de servicio/2/Compras en tienda,additionalInfo/Opciones de servicio/2/Entrega el mismo día,additionalInfo/Opciones de servicio/2/Para llevar,additionalInfo/Opciones de servicio/2/Retiro en tienda,additionalInfo/Opciones de servicio/3/Compras en tienda,additionalInfo/Opciones de servicio/3/Entrega el mismo día,additionalInfo/Opciones de servicio/4/Entrega el mismo día,additionalInfo/Pagos/0/Cheques,additionalInfo/Pagos/0/Pagos móviles mediante NFC,additionalInfo/Pagos/0/Solo dinero en efectivo,additionalInfo/Pagos/0/Tarjetas de crédito,additionalInfo/Pagos/0/Tarjetas de débito,additionalInfo/Pagos/1/Pagos móviles mediante NFC,additionalInfo/Pagos/1/Solo dinero en efectivo,additionalInfo/Pagos/1/Tarjetas de crédito,additionalInfo/Pagos/1/Tarjetas de débito,additionalInfo/Pagos/2/Tarjetas de crédito,additionalInfo/Pagos/2/Tarjetas de débito,additionalInfo/Pagos/3/Tarjetas de crédito,additionalInfo/Pagos/3/Tarjetas de débito,additionalInfo/Pagos/4/Tarjetas de crédito,additionalInfo/Planificación/0/Se requiere una cita,additionalInfo/Planificación/0/Visita rápida,additionalInfo/Público usual/0/Amigable con LGBTQ+,additionalInfo/Público usual/1/Espacio seguro para personas transgénero,additionalInfo/Qué ofrece/0/Combustible diésel,additionalInfo/Qué ofrece/0/Comida,additionalInfo/Qué ofrece/0/Comida preparada,additionalInfo/Qué ofrece/0/Servicio de armado,additionalInfo/Qué ofrece/1/Gasolinera con servicio completo,additionalInfo/Qué ofrece/2/Intercambio de tanques de propano,additionalInfo/Qué ofrece/3/Lavadero de autos,additionalInfo/Reciclaje/0/Baterías,additionalInfo/Reciclaje/0/Dispositivos electrónicos,additionalInfo/Reciclaje/1/Bombillas,additionalInfo/Servicios/0/Minimercado,additionalInfo/Servicios/0/Sanitarios unisex,additionalInfo/Servicios/0/Wi-Fi,additionalInfo/Servicios/1/Sanitario,additionalInfo/Servicios/1/Tiene Wi-Fi gratis,additionalInfo/Servicios/1/Wi-Fi,additionalInfo/Servicios/2/Tiene Wi-Fi gratis,address,categories/0,categories/1,categories/2,categories/3,categoryName,cid,city,claimThisBusiness,countryCode,description,fid,googleFoodUrl,imageUrl,imagesCount,isAdvertisement,kgmid,language,location/lat,location/lng,neighborhood,openingHours/0/day,openingHours/0/hours,openingHours/1/day,openingHours/1/hours,openingHours/2/day,openingHours/2/hours,openingHours/

In [62]:
df_rauch_03.shape

(166, 124)

In [63]:
df_rauch_03 = df_rauch_03[[
        'title',
        'categoryName',
        'website',
        'street',
        'phone',
        'city',
        'state',
        'location/lat', 
        'location/lng',
        'totalScore',
        'rank',
        'imageUrl',
        'imagesCount',       
        'reviewsCount', 
        'scrapedAt',
        'searchString']]
df_rauch_03.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,"Despensa y anexo belen ""lo de jhonny""",Comercio,NaN,Barrio Flucho Casa 23,+54 9 249 448-5829,Rauch,Provincia de Buenos Aires,-36.780949,-59.078725,5.0,1,https://lh3.googleusercontent.com/p/AF1QipN4p-...,33.0,9,2025-05-19T00:05:29.686Z,Comercio
1,El Viejo Almacén,Comercio,NaN,NaN,+54 2297 44-2351,Rauch,Provincia de Buenos Aires,-36.777091,-59.088512,4.5,2,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,2,2025-05-19T00:05:29.688Z,Comercio
2,Campos Julian - Contenedores,Comercio,NaN,Av. Belgrano 573,+54 2297 44-2714,Rauch,Provincia de Buenos Aires,-36.779640,-59.081616,4.8,3,https://lh3.googleusercontent.com/gps-cs-s/AC9...,3.0,4,2025-05-19T00:05:29.688Z,Comercio
3,Arturo E Dualde,Comercio,NaN,Av. San Martín 1060,+54 2297 44-0384,Rauch,Provincia de Buenos Aires,-36.771673,-59.085032,4.5,4,https://lh3.googleusercontent.com/gps-cs-s/AC9...,3.0,2,2025-05-19T00:05:29.688Z,Comercio
4,Miguel A Siervo,Comercio,NaN,"ABB, 9 de Julio 62",+54 2297 44-0451,Rauch,Provincia de Buenos Aires,-36.776702,-59.092649,5.0,5,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,1,2025-05-19T00:05:29.691Z,Comercio


In [64]:
df_rauch_03.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166 entries, 0 to 165
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         166 non-null    object 
 1   categoryName  163 non-null    object 
 2   website       27 non-null     object 
 3   street        150 non-null    object 
 4   phone         87 non-null     object 
 5   city          165 non-null    object 
 6   state         165 non-null    object 
 7   location/lat  166 non-null    float64
 8   location/lng  166 non-null    float64
 9   totalScore    108 non-null    float64
 10  rank          166 non-null    int64  
 11  imageUrl      165 non-null    object 
 12  imagesCount   165 non-null    float64
 13  reviewsCount  166 non-null    int64  
 14  scrapedAt     166 non-null    object 
 15  searchString  166 non-null    object 
dtypes: float64(4), int64(2), object(10)
memory usage: 20.9+ KB


In [65]:
df_rauch_03.to_csv('../data/processed/01_dataPrep/rauch/03_rauch.csv',  index = False)

In [66]:
glob.glob('../data/processed/01_dataPrep/rauch/*.csv')

['../data/processed/01_dataPrep/rauch/01_rauch.csv',
 '../data/processed/01_dataPrep/rauch/02_rauch.csv',
 '../data/processed/01_dataPrep/rauch/03_rauch.csv']

In [67]:
df = pd.concat(map(pd.read_csv, glob.glob('../data/processed/01_dataPrep/rauch/*.csv')))
df.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Del Valle inyeccion electronica,Taller de reparación de automóviles,NaN,"Aristóbulo del Valle, Aveleyra &",+54 249 434-0861,Rauch,Provincia de Buenos Aires,-36.773298,-59.091459,5.0,1,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,1,2025-05-29T00:48:32.392Z,electronica
1,Electro 440 - Accesorios y Reparación,Tienda de electrónica,https://www.facebook.com/todoenremotos,Letamendi 596,+54 9 249 427-7096,Rauch,Provincia de Buenos Aires,-36.776094,-59.088072,5.0,2,https://lh3.googleusercontent.com/gps-cs-s/AC9...,35.0,5,2025-05-29T00:48:32.394Z,electronica
2,Dangerous Sonido e Iluminación,Tienda de equipos de sonido,NaN,"BNK, Saavedra 507",+54 249 453-3029,Rauch,Provincia de Buenos Aires,-36.772975,-59.093649,5.0,3,https://lh3.googleusercontent.com/gps-cs-s/AC9...,28.0,3,2025-05-29T00:48:32.394Z,electronica
3,Jona PC,Servicio de informática,https://www.facebook.com/profile.php?id=100069...,Dardo Rocha 850,NaN,Rauch,Provincia de Buenos Aires,-36.761649,-59.092823,5.0,4,https://lh3.googleusercontent.com/p/AF1QipPcWb...,13.0,11,2025-05-29T00:48:32.394Z,electronica
4,Super PC Rauch,Tienda de informática,NaN,Balcarce 1001,+54 249 435-8292,Rauch,Provincia de Buenos Aires,-36.768249,-59.090148,NaN,5,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,0,2025-05-29T00:48:32.395Z,electronica


In [68]:
df.to_csv('../data/processed/01_dataPrep/main/main_rauch.csv', index=False)

In [69]:
df_tandil_01 = pd.read_csv('../data/raw/tandil _dataset_google-maps-extractor_2025-05-04_16-09-01-070.csv')
df_tandil_01.head(5)

,additionalInfo/Accesibilidad/0/Entrada accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Bucle magnético de asistencia,additionalInfo/Accesibilidad/1/Entrada accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/2/Bucle magnético de asistencia,additionalInfo/Accesibilidad/2/Entrada accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/2/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/2/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/2/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/3/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/3/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/4/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Ambiente/0/A la moda,additionalInfo/Ambiente/0/Agradable,additionalInfo/Ambiente/1/Agradable,additionalInfo/Ambiente/1/Informal,additionalInfo/Ambiente/2/Informal,additionalInfo/Ambiente/2/Relajado,additionalInfo/Ambiente/3/Relajado,additionalInfo/Amenities/0/Wi-Fi gratis,additionalInfo/Aspectos destacados/0/Buen café,additionalInfo/Aspectos destacados/0/Buenos productos frescos,additionalInfo/Aspectos destacados/0/Deliciosos postres,additionalInfo/Aspectos destacados/1/Deliciosos postres,additionalInfo/Aspectos destacados/2/Deportes,additionalInfo/Aspectos destacados/2/Muy buena selección de cervezas,additionalInfo/Aspectos destacados/2/Muy buena selección de té,additionalInfo/Aspectos destacados/3/Muy buena selección de cervezas,additionalInfo/Aspectos destacados/3/Muy buena selección de té,additionalInfo/Aspectos destacados/4/Muy buena selección de té,additionalInfo/Estacionamiento/0,additionalInfo/Estacionamiento/0/Estacionamiento en el lugar,additionalInfo/Estacionamiento/0/Estacionamiento gratuito,additionalInfo/Estacionamiento/0/Estacionamiento gratuito en la calle,additionalInfo/Estacionamiento/0/Garage de stacionamiento gratuito,additionalInfo/Estacionamiento/1/Estacionamiento gratuito,additionalInfo/Estacionamiento/1/Estacionamiento gratuito en la calle,additionalInfo/Estacionamiento/1/Estacionamiento pagado,additionalInfo/Estacionamiento/1/Estacionamiento pagado en la calle,additionalInfo/Estacionamiento/2/Estacionamiento gratuito en la calle,additionalInfo/Estacionamiento/2/Estacionamiento pagado en la calle,additionalInfo/Estacionamiento/3/Garage de stacionamiento gratuito,additionalInfo/Información proporcionada por la empresa/0/Se identifica como mujer empresaria,additionalInfo/Mascotas/0/Se permiten perros,additionalInfo/Mascotas/1/Se permiten perros afuera,additionalInfo/Menores/0/Ideal para ir con niños,additionalInfo/Menores/1/Menú para niños,additionalInfo/Menores/2/Sillas altas,additionalInfo/Opciones de servicio/0/Asientos al aire libre,additionalInfo/Opciones de servicio/0/Clases en línea,additionalInfo/Opciones de servicio/0/Compras en tienda,additionalInfo/Opciones de servicio/0/Consumo en el lugar,additionalInfo/Opciones de servicio/0/Entrega a domicilio,additionalInfo/Opciones de servicio/0/Entrega el mismo día,additionalInfo/Opciones de servicio/0/Entrega sin contacto,additionalInfo/Opciones de servicio/0/Para llevar,additionalInfo/Opciones de servicio/0/Retiro en tienda,additionalInfo/Opciones de servicio/0/Retiros en la puerta,additionalInfo/Opciones de servicio/0/Servicios en el lugar,additionalInfo/Opciones

In [70]:
df_tandil_01.shape

(600, 353)

In [71]:
df_tandil_01 = df_tandil_01[[
        'title',
        'categoryName',
        'website',
        'street',
        'phone',
        'city',
        'state',
        'location/lat', 
        'location/lng',
        'totalScore',
        'rank',
        'imageUrl',
        'imagesCount',       
        'reviewsCount', 
        'scrapedAt',
        'searchString']]
df_tandil_01.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,EL BAQUEANO TANDIL,Tienda de delicatessen,http://www.elbaqueanotandil.com/,"1381, Tasende",NaN,Tandil,Provincia de Buenos Aires,-37.350908,-59.170069,4.8,8,https://lh3.googleusercontent.com/p/AF1QipP2-l...,686.0,1408,2025-05-04T16:08:15.927Z,Comercio
1,Alquimia Artesanías,Tienda de artesanías,https://www.facebook.com/alquimia.artesanias.t...,Icalma y Tasende Acceso Cerro El Centinela,NaN,Tandil,Provincia de Buenos Aires,-37.355833,-59.172222,4.5,9,https://lh3.googleusercontent.com/p/AF1QipPXrh...,40.0,43,2025-05-04T16:08:15.927Z,Comercio
2,mercado Jorge,Tienda de alimentación,NaN,Sandino 468,+54 249 444-3579,Tandil,Provincia de Buenos Aires,-37.338040,-59.126114,4.5,10,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,13,2025-05-04T16:08:15.927Z,Comercio
3,"Vale la Pena ""Almacén""",Tienda de alimentación,NaN,Av. Estrada 468,NaN,Tandil,Provincia de Buenos Aires,-37.339490,-59.147503,4.6,11,https://lh3.googleusercontent.com/p/AF1QipPz_6...,3.0,17,2025-05-04T16:08:15.927Z,Comercio
4,Sec Tandil - Sociedad Emp de Comercio Tandil,Sindicato,http://www.sectandil.org.ar/,San Martín 785,NaN,Tandil,Provincia de Buenos Aires,-37.325458,-59.134093,1.0,12,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,2,2025-05-04T16:08:15.927Z,Comercio


In [72]:
df_tandil_01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         600 non-null    object 
 1   categoryName  598 non-null    object 
 2   website       159 non-null    object 
 3   street        571 non-null    object 
 4   phone         389 non-null    object 
 5   city          594 non-null    object 
 6   state         596 non-null    object 
 7   location/lat  600 non-null    float64
 8   location/lng  600 non-null    float64
 9   totalScore    460 non-null    float64
 10  rank          600 non-null    int64  
 11  imageUrl      586 non-null    object 
 12  imagesCount   586 non-null    float64
 13  reviewsCount  600 non-null    int64  
 14  scrapedAt     600 non-null    object 
 15  searchString  600 non-null    object 
dtypes: float64(4), int64(2), object(10)
memory usage: 75.1+ KB


In [73]:
df_tandil_01.to_csv('../data/processed/01_dataPrep/tandil/01_tandil.csv', index=False)

In [74]:
df_tandil_02 = pd.read_csv('../data/raw/tandil_dataset_crawler-google-places_2025-05-28_23-29-20-035.csv')
df_tandil_02.head(5)

,additionalInfo/Accesibilidad/0/Entrada accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Aspectos destacados/0/Buenos productos frescos,additionalInfo/Estacionamiento/0/Estacionamiento en el lugar,additionalInfo/Estacionamiento/1/Estacionamiento gratuito,additionalInfo/Estacionamiento/2/Estacionamiento gratuito en la calle,additionalInfo/Opciones de servicio/0/Citas en línea,additionalInfo/Opciones de servicio/0/Entrega a domicilio,additionalInfo/Opciones de servicio/0/Entrega el mismo día,additionalInfo/Opciones de servicio/0/Entrega sin contacto,additionalInfo/Opciones de servicio/0/Retiros en la puerta,additionalInfo/Opciones de servicio/0/Servicios en el lugar,additionalInfo/Opciones de servicio/1/Compras en tienda,additionalInfo/Opciones de servicio/1/Entrega a domicilio,additionalInfo/Opciones de servicio/1/Entrega el mismo día,additionalInfo/Opciones de servicio/1/Entrega sin contacto,additionalInfo/Opciones de servicio/1/Retiro en tienda,additionalInfo/Opciones de servicio/2/Compras en tienda,additionalInfo/Opciones de servicio/2/Entrega a domicilio,additionalInfo/Opciones de servicio/2/Entrega el mismo día,additionalInfo/Opciones de servicio/2/Retiro en tienda,additionalInfo/Opciones de servicio/2/Servicios en el lugar,additionalInfo/Opciones de servicio/3/Compras en tienda,additionalInfo/Opciones de servicio/3/Entrega el mismo día,additionalInfo/Opciones de servicio/3/Retiro en tienda,additionalInfo/Opciones de servicio/3/Servicios en el lugar,additionalInfo/Opciones de servicio/4/Citas en línea,additionalInfo/Opciones de servicio/4/Compras en tienda,additionalInfo/Opciones de servicio/4/Entrega el mismo día,additionalInfo/Opciones de servicio/4/Servicios en el lugar,additionalInfo/Opciones de servicio/5/Entrega el mismo día,additionalInfo/Pagos/0/Cheques,additionalInfo/Pagos/0/Pagos móviles mediante NFC,additionalInfo/Pagos/0/Tarjetas de crédito,additionalInfo/Pagos/0/Tarjetas de débito,additionalInfo/Pagos/1/Pagos móviles mediante NFC,additionalInfo/Pagos/1/Tarjetas de crédito,additionalInfo/Pagos/1/Tarjetas de débito,additionalInfo/Pagos/2/Tarjetas de crédito,additionalInfo/Pagos/2/Tarjetas de débito,additionalInfo/Pagos/3/Tarjetas de crédito,additionalInfo/Pagos/3/Tarjetas de débito,additionalInfo/Pagos/4/Tarjetas de crédito,additionalInfo/Planificación/0/Se recomienda concertar cita,additionalInfo/Planificación/0/Se requiere una cita,additionalInfo/Planificación/0/Visita rápida,additionalInfo/Público usual/0/Amigable con LGBTQ+,additionalInfo/Público usual/1/Espacio seguro para personas transgénero,additionalInfo/Qué ofrece/0/Comida,additionalInfo/Qué ofrece/0/Servicio de armado,additionalInfo/Qué ofrece/0/Servicios de reparación,additionalInfo/Qué ofrece/1/Comida preparada,additionalInfo/Qué ofrece/1/Productos orgánicos,additionalInfo/Qué ofrece/1/Servicios de reparación,additionalInfo/Qué ofrece/2/Productos orgánicos,additionalInfo/Reciclaje/0/Baterías,additionalInfo/Reciclaje/0/Dispositivos electrónicos,additionalInfo/Servicios/0/Sanitario,additionalInfo/Servicios/0/Sanitarios unisex,additionalInfo/Servicios/0/Wi-Fi,additionalInfo/Servicios/1/Tiene Wi-Fi gratis,additionalInfo/Servicios/1/Tiene Wi-Fi pago,additionalInfo/Servicios/1/Wi-Fi,additionalInfo/Servicios/2/Tiene Wi-Fi gratis,additionalOpeningHours/Horario de atención en línea/0/day,additionalOpeningHours/Horario de atención en línea/0/hours,additionalOpeningHours/Horario de atención en línea/1/day,additionalOpeningHours/Horario de atención en línea/1/hours,additionalOpeningHours/Horario de atención en línea/2/day,additionalOpeningHours/Horario de atención en línea/2/hours,additionalOpeningHours/Horario de atención en línea/3/day,additionalOpeningHours/Horario de atención en línea/3/hour

In [75]:
df_tandil_02.shape

(50, 156)

In [76]:
df_tandil_02 = df_tandil_02[[
        'title',
        'categoryName',
        'website',
        'street',
        'phone',
        'city',
        'state',
        'location/lat', 
        'location/lng',
        'totalScore',
        'rank',
        'imageUrl',
        'imagesCount',       
        'reviewsCount', 
        'scrapedAt',
        'searchString']]
df_tandil_02.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Electronica Montiel,Taller mecánico,NaN,Colectora Nte. L. M. Macaya 1055,+54 249 434-0599,Tandil,Provincia de Buenos Aires,-37.310936,-59.114156,4.8,1,https://lh3.googleusercontent.com/p/AF1QipNzII...,25.0,24,2025-05-28T23:28:48.648Z,"electronica, tienda"
1,Roqui Electronica,Tienda de electrónica,http://www.facebook.com/roqui.electronica,Gral. Rodríguez 1316,+54 249 457-3257,Tandil,Provincia de Buenos Aires,-37.317827,-59.141964,4.3,2,https://lh3.googleusercontent.com/p/AF1QipMF8N...,313.0,673,2025-05-28T23:28:48.649Z,"electronica, tienda"
2,RURALINK,Tienda de electrónica,http://ruralink.com.ar/,Av. Buzón 727,+54 249 430-2244,BWH,Provincia de Buenos Aires,-37.320178,-59.118276,2.0,3,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,52,2025-05-28T23:28:48.649Z,"electronica, tienda"
3,Escapes Daniel Celentano,Tienda de electrónica,NaN,"399,, Primera Junta 299",+54 9 249 467-3486,Tandil,Provincia de Buenos Aires,-37.319884,-59.108358,4.7,4,https://lh3.googleusercontent.com/gps-cs-s/AC9...,3.0,48,2025-05-28T23:28:48.650Z,"electronica, tienda"
4,Mixer electronicos,Tienda de accesorios para móviles,NaN,Darragueira 2216,+54 9 249 436-1450,Tandil,Provincia de Buenos Aires,-37.298917,-59.125984,4.2,5,https://lh3.googleusercontent.com/gps-cs-s/AC9...,6.0,6,2025-05-28T23:28:48.650Z,"electronica, tienda"


In [77]:
df_tandil_02.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         50 non-null     object 
 1   categoryName  46 non-null     object 
 2   website       28 non-null     object 
 3   street        49 non-null     object 
 4   phone         45 non-null     object 
 5   city          50 non-null     object 
 6   state         50 non-null     object 
 7   location/lat  50 non-null     float64
 8   location/lng  50 non-null     float64
 9   totalScore    41 non-null     float64
 10  rank          50 non-null     int64  
 11  imageUrl      48 non-null     object 
 12  imagesCount   48 non-null     float64
 13  reviewsCount  50 non-null     int64  
 14  scrapedAt     50 non-null     object 
 15  searchString  50 non-null     object 
dtypes: float64(4), int64(2), object(10)
memory usage: 6.4+ KB


In [78]:
df_tandil_02.to_csv('../data/processed/01_dataPrep/tandil/02_tandil.csv', index=False)

In [79]:
df_tandil_03 = pd.read_csv('../data/raw/tandil_dataset_crawler-google-places_2025-05-28_23-37-49-506.csv')
df_tandil_03.head(5)

,additionalInfo/Accesibilidad/0/Entrada accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Aspectos destacados/0/Buenos productos frescos,additionalInfo/Aspectos destacados/0/Películas 3D,additionalInfo/Estacionamiento/0/Estacionamiento en el lugar,additionalInfo/Estacionamiento/1/Estacionamiento gratuito,additionalInfo/Estacionamiento/2/Estacionamiento gratuito en la calle,additionalInfo/Menores/0/Ideal para ir con niños,additionalInfo/Opciones de servicio/0/Citas en línea,additionalInfo/Opciones de servicio/0/Entrega a domicilio,additionalInfo/Opciones de servicio/0/Entrega el mismo día,additionalInfo/Opciones de servicio/0/Entrega sin contacto,additionalInfo/Opciones de servicio/0/Retiros en la puerta,additionalInfo/Opciones de servicio/0/Servicios en el lugar,additionalInfo/Opciones de servicio/1/Compras en tienda,additionalInfo/Opciones de servicio/1/Entrega a domicilio,additionalInfo/Opciones de servicio/1/Entrega el mismo día,additionalInfo/Opciones de servicio/1/Entrega sin contacto,additionalInfo/Opciones de servicio/1/Retiro en tienda,additionalInfo/Opciones de servicio/1/Servicios en el lugar,additionalInfo/Opciones de servicio/2/Compras en tienda,additionalInfo/Opciones de servicio/2/Entrega a domicilio,additionalInfo/Opciones de servicio/2/Entrega el mismo día,additionalInfo/Opciones de servicio/2/Retiro en tienda,additionalInfo/Opciones de servicio/2/Servicios en el lugar,additionalInfo/Opciones de servicio/3/Compras en tienda,additionalInfo/Opciones de servicio/3/Entrega el mismo día,additionalInfo/Opciones de servicio/3/Retiro en tienda,additionalInfo/Opciones de servicio/3/Servicios en el lugar,additionalInfo/Opciones de servicio/4/Citas en línea,additionalInfo/Opciones de servicio/4/Compras en tienda,additionalInfo/Opciones de servicio/4/Entrega el mismo día,additionalInfo/Opciones de servicio/4/Servicios en el lugar,additionalInfo/Opciones de servicio/5/Entrega el mismo día,additionalInfo/Pagos/0/Cheques,additionalInfo/Pagos/0/Pagos móviles mediante NFC,additionalInfo/Pagos/0/Tarjetas de crédito,additionalInfo/Pagos/0/Tarjetas de débito,additionalInfo/Pagos/1/Pagos móviles mediante NFC,additionalInfo/Pagos/1/Tarjetas de crédito,additionalInfo/Pagos/1/Tarjetas de débito,additionalInfo/Pagos/2/Tarjetas de crédito,additionalInfo/Pagos/2/Tarjetas de débito,additionalInfo/Pagos/3/Tarjetas de crédito,additionalInfo/Pagos/3/Tarjetas de débito,additionalInfo/Pagos/4/Tarjetas de crédito,additionalInfo/Planificación/0/Se recomienda concertar cita,additionalInfo/Planificación/0/Se requiere una cita,additionalInfo/Planificación/0/Visita rápida,additionalInfo/Público usual/0/Amigable con LGBTQ+,additionalInfo/Público usual/0/Espacio seguro para personas transgénero,additionalInfo/Público usual/1/Espacio seguro para personas transgénero,additionalInfo/Qué ofrece/0/Comida,additionalInfo/Qué ofrece/0/Servicio de armado,additionalInfo/Qué ofrece/0/Servicios de reparación,additionalInfo/Qué ofrece/1/Comida preparada,additionalInfo/Qué ofrece/1/Juegos de arcade,additionalInfo/Qué ofrece/1/Productos orgánicos,additionalInfo/Qué ofrece/1/Servicios de reparación,additionalInfo/Qué ofrece/2/Productos orgánicos,additionalInfo/Reciclaje/0/Baterías,additionalInfo/Reciclaje/0/Dispositivos electrónicos,additionalInfo/Servicios/0/Restaurante,additionalInfo/Servicios/0/Sanitario,additionalInfo/Servicios/0/Sanitarios unisex,additionalInfo/Servicios/0/Taller mecánico,additionalInfo/Servicios/0/Wi-Fi,additionalInfo/Servicios/1/Tiene Wi-Fi gratis,additionalInfo/Servicios/1/Tiene Wi-Fi pago,additionalInfo/Servicios/1/Wi-Fi,additionalInfo/Servicios/2/Tiene Wi-Fi gratis,additionalOpeningHours/Hor

In [80]:
df_tandil_03.shape

(83, 332)

In [81]:
df_tandil_03 = df_tandil_03[[
        'title',
        'categoryName',
        'website',
        'street',
        'phone',
        'city',
        'state',
        'location/lat', 
        'location/lng',
        'totalScore',
        'rank',
        'imageUrl',
        'imagesCount',       
        'reviewsCount', 
        'scrapedAt',
        'searchString']]
df_tandil_03.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Vitalia Computación,Tienda de informática,NaN,Quintana 595,+54 249 420-7361,Tandil,Provincia de Buenos Aires,-37.307385,-59.142590,4.3,17,https://lh3.googleusercontent.com/p/AF1QipMwwD...,9.0,24,2025-05-28T23:34:37.831Z,electronica
1,"JCD Julio Cesar Díaz , Fotografía, Electrónica...",Fotógrafo,http://juliocesardiaz.ph/instagram,Panamá 351,+54 249 451-0029,Tandil,Provincia de Buenos Aires,-37.324125,-59.114565,4.0,16,https://lh3.googleusercontent.com/p/AF1QipMGUd...,39.0,21,2025-05-28T23:34:37.831Z,electronica
2,Taller de TV y Electronica LeoTV,Establecimiento de reparación de artículos ele...,http://www.serviceleotv.com.ar/,Gral. Rodríguez 1260,+54 249 452-1111,Tandil,Provincia de Buenos Aires,-37.318632,-59.141470,4.1,11,https://lh3.googleusercontent.com/p/AF1QipOoRM...,28.0,79,2025-05-28T23:34:37.831Z,electronica
3,Zangar Electrónica Sistemas de Seguridad Hogar...,NaN,http://www.zangarelectronica.com.ar/,14 de Julio 770,+54 249 442-6636,Tandil,Provincia de Buenos Aires,-37.325971,-59.141926,4.1,9,https://lh3.googleusercontent.com/gps-cs-s/AC9...,15.0,44,2025-05-28T23:34:37.827Z,electronica
4,supercell,Tienda de electrónica,NaN,Av. Perón,NaN,Tandil,Provincia de Buenos Aires,-37.321193,-59.149952,5.0,20,NaN,NaN,1,2025-05-28T23:34:37.251Z,electronica


In [82]:
df_tandil_03.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         83 non-null     object 
 1   categoryName  75 non-null     object 
 2   website       48 non-null     object 
 3   street        80 non-null     object 
 4   phone         70 non-null     object 
 5   city          82 non-null     object 
 6   state         83 non-null     object 
 7   location/lat  83 non-null     float64
 8   location/lng  83 non-null     float64
 9   totalScore    65 non-null     float64
 10  rank          83 non-null     int64  
 11  imageUrl      80 non-null     object 
 12  imagesCount   80 non-null     float64
 13  reviewsCount  83 non-null     int64  
 14  scrapedAt     83 non-null     object 
 15  searchString  83 non-null     object 
dtypes: float64(4), int64(2), object(10)
memory usage: 10.5+ KB


In [83]:
df_tandil_03.to_csv('../data/processed/01_dataPrep/tandil/03_tandil.csv', index=False)

In [84]:
df_tandil_04 = pd.read_csv('../data/raw/tandil_dataset_crawler-google-places_2025-05-28_23-37-49-506.csv')
df_tandil_04.head(5)

,additionalInfo/Accesibilidad/0/Entrada accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Aspectos destacados/0/Buenos productos frescos,additionalInfo/Aspectos destacados/0/Películas 3D,additionalInfo/Estacionamiento/0/Estacionamiento en el lugar,additionalInfo/Estacionamiento/1/Estacionamiento gratuito,additionalInfo/Estacionamiento/2/Estacionamiento gratuito en la calle,additionalInfo/Menores/0/Ideal para ir con niños,additionalInfo/Opciones de servicio/0/Citas en línea,additionalInfo/Opciones de servicio/0/Entrega a domicilio,additionalInfo/Opciones de servicio/0/Entrega el mismo día,additionalInfo/Opciones de servicio/0/Entrega sin contacto,additionalInfo/Opciones de servicio/0/Retiros en la puerta,additionalInfo/Opciones de servicio/0/Servicios en el lugar,additionalInfo/Opciones de servicio/1/Compras en tienda,additionalInfo/Opciones de servicio/1/Entrega a domicilio,additionalInfo/Opciones de servicio/1/Entrega el mismo día,additionalInfo/Opciones de servicio/1/Entrega sin contacto,additionalInfo/Opciones de servicio/1/Retiro en tienda,additionalInfo/Opciones de servicio/1/Servicios en el lugar,additionalInfo/Opciones de servicio/2/Compras en tienda,additionalInfo/Opciones de servicio/2/Entrega a domicilio,additionalInfo/Opciones de servicio/2/Entrega el mismo día,additionalInfo/Opciones de servicio/2/Retiro en tienda,additionalInfo/Opciones de servicio/2/Servicios en el lugar,additionalInfo/Opciones de servicio/3/Compras en tienda,additionalInfo/Opciones de servicio/3/Entrega el mismo día,additionalInfo/Opciones de servicio/3/Retiro en tienda,additionalInfo/Opciones de servicio/3/Servicios en el lugar,additionalInfo/Opciones de servicio/4/Citas en línea,additionalInfo/Opciones de servicio/4/Compras en tienda,additionalInfo/Opciones de servicio/4/Entrega el mismo día,additionalInfo/Opciones de servicio/4/Servicios en el lugar,additionalInfo/Opciones de servicio/5/Entrega el mismo día,additionalInfo/Pagos/0/Cheques,additionalInfo/Pagos/0/Pagos móviles mediante NFC,additionalInfo/Pagos/0/Tarjetas de crédito,additionalInfo/Pagos/0/Tarjetas de débito,additionalInfo/Pagos/1/Pagos móviles mediante NFC,additionalInfo/Pagos/1/Tarjetas de crédito,additionalInfo/Pagos/1/Tarjetas de débito,additionalInfo/Pagos/2/Tarjetas de crédito,additionalInfo/Pagos/2/Tarjetas de débito,additionalInfo/Pagos/3/Tarjetas de crédito,additionalInfo/Pagos/3/Tarjetas de débito,additionalInfo/Pagos/4/Tarjetas de crédito,additionalInfo/Planificación/0/Se recomienda concertar cita,additionalInfo/Planificación/0/Se requiere una cita,additionalInfo/Planificación/0/Visita rápida,additionalInfo/Público usual/0/Amigable con LGBTQ+,additionalInfo/Público usual/0/Espacio seguro para personas transgénero,additionalInfo/Público usual/1/Espacio seguro para personas transgénero,additionalInfo/Qué ofrece/0/Comida,additionalInfo/Qué ofrece/0/Servicio de armado,additionalInfo/Qué ofrece/0/Servicios de reparación,additionalInfo/Qué ofrece/1/Comida preparada,additionalInfo/Qué ofrece/1/Juegos de arcade,additionalInfo/Qué ofrece/1/Productos orgánicos,additionalInfo/Qué ofrece/1/Servicios de reparación,additionalInfo/Qué ofrece/2/Productos orgánicos,additionalInfo/Reciclaje/0/Baterías,additionalInfo/Reciclaje/0/Dispositivos electrónicos,additionalInfo/Servicios/0/Restaurante,additionalInfo/Servicios/0/Sanitario,additionalInfo/Servicios/0/Sanitarios unisex,additionalInfo/Servicios/0/Taller mecánico,additionalInfo/Servicios/0/Wi-Fi,additionalInfo/Servicios/1/Tiene Wi-Fi gratis,additionalInfo/Servicios/1/Tiene Wi-Fi pago,additionalInfo/Servicios/1/Wi-Fi,additionalInfo/Servicios/2/Tiene Wi-Fi gratis,additionalOpeningHours/Hor

In [85]:
df_tandil_04.shape

(83, 332)

In [86]:
df_tandil_04 = df_tandil_04[[
        'title',
        'categoryName',
        'website',
        'street',
        'phone',
        'city',
        'state',
        'location/lat', 
        'location/lng',
        'totalScore',
        'rank',
        'imageUrl',
        'imagesCount',       
        'reviewsCount', 
        'scrapedAt',
        'searchString']]
df_tandil_04.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Vitalia Computación,Tienda de informática,NaN,Quintana 595,+54 249 420-7361,Tandil,Provincia de Buenos Aires,-37.307385,-59.142590,4.3,17,https://lh3.googleusercontent.com/p/AF1QipMwwD...,9.0,24,2025-05-28T23:34:37.831Z,electronica
1,"JCD Julio Cesar Díaz , Fotografía, Electrónica...",Fotógrafo,http://juliocesardiaz.ph/instagram,Panamá 351,+54 249 451-0029,Tandil,Provincia de Buenos Aires,-37.324125,-59.114565,4.0,16,https://lh3.googleusercontent.com/p/AF1QipMGUd...,39.0,21,2025-05-28T23:34:37.831Z,electronica
2,Taller de TV y Electronica LeoTV,Establecimiento de reparación de artículos ele...,http://www.serviceleotv.com.ar/,Gral. Rodríguez 1260,+54 249 452-1111,Tandil,Provincia de Buenos Aires,-37.318632,-59.141470,4.1,11,https://lh3.googleusercontent.com/p/AF1QipOoRM...,28.0,79,2025-05-28T23:34:37.831Z,electronica
3,Zangar Electrónica Sistemas de Seguridad Hogar...,NaN,http://www.zangarelectronica.com.ar/,14 de Julio 770,+54 249 442-6636,Tandil,Provincia de Buenos Aires,-37.325971,-59.141926,4.1,9,https://lh3.googleusercontent.com/gps-cs-s/AC9...,15.0,44,2025-05-28T23:34:37.827Z,electronica
4,supercell,Tienda de electrónica,NaN,Av. Perón,NaN,Tandil,Provincia de Buenos Aires,-37.321193,-59.149952,5.0,20,NaN,NaN,1,2025-05-28T23:34:37.251Z,electronica


In [87]:
df_tandil_04.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         83 non-null     object 
 1   categoryName  75 non-null     object 
 2   website       48 non-null     object 
 3   street        80 non-null     object 
 4   phone         70 non-null     object 
 5   city          82 non-null     object 
 6   state         83 non-null     object 
 7   location/lat  83 non-null     float64
 8   location/lng  83 non-null     float64
 9   totalScore    65 non-null     float64
 10  rank          83 non-null     int64  
 11  imageUrl      80 non-null     object 
 12  imagesCount   80 non-null     float64
 13  reviewsCount  83 non-null     int64  
 14  scrapedAt     83 non-null     object 
 15  searchString  83 non-null     object 
dtypes: float64(4), int64(2), object(10)
memory usage: 10.5+ KB


In [88]:
df_tandil_04.to_csv('../data/processed/01_dataPrep/tandil/04_tandil.csv', index=False)

In [89]:
df_tandil_05 = pd.read_csv('../data/raw/tandil_dataset_crawler-google-places_2025-05-28_23-44-51-919.csv')
df_tandil_05.head(5)

,additionalInfo/Accesibilidad/0/Entrada accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Ambiente/0/A la moda,additionalInfo/Ambiente/1/Agradable,additionalInfo/Ambiente/2/Informal,additionalInfo/Ambiente/3/Relajado,additionalInfo/Aspectos destacados/0/Buenos productos frescos,additionalInfo/Aspectos destacados/0/Deliciosos postres,additionalInfo/Aspectos destacados/0/Muy buena selección de cervezas,additionalInfo/Aspectos destacados/1/Deportes,additionalInfo/Estacionamiento/0/Estacionamiento gratuito en la calle,additionalInfo/Información proporcionada por la empresa/0/Se identifica como mujer empresaria,additionalInfo/Menores/0/Ideal para ir con niños,additionalInfo/Menores/1/Tiene cambiadores de pañales,additionalInfo/Opciones de servicio/0/Asientos al aire libre,additionalInfo/Opciones de servicio/0/Citas en línea,additionalInfo/Opciones de servicio/0/Compras en tienda,additionalInfo/Opciones de servicio/0/Entrega a domicilio,additionalInfo/Opciones de servicio/0/Retiros en la puerta,additionalInfo/Opciones de servicio/0/Servicios en el lugar,additionalInfo/Opciones de servicio/1/Citas en línea,additionalInfo/Opciones de servicio/1/Entrega a domicilio,additionalInfo/Opciones de servicio/1/Entrega el mismo día,additionalInfo/Opciones de servicio/1/Entrega sin contacto,additionalInfo/Opciones de servicio/1/Retiro en tienda,additionalInfo/Opciones de servicio/1/Retiros en la puerta,additionalInfo/Opciones de servicio/1/Servicios en el lugar,additionalInfo/Opciones de servicio/2/Compras en tienda,additionalInfo/Opciones de servicio/2/Entrega a domicilio,additionalInfo/Opciones de servicio/2/Entrega el mismo día,additionalInfo/Opciones de servicio/2/Retiro en tienda,additionalInfo/Opciones de servicio/2/Servicios en el lugar,additionalInfo/Opciones de servicio/3/Compras en tienda,additionalInfo/Opciones de servicio/3/Entrega el mismo día,additionalInfo/Opciones de servicio/3/Para llevar,additionalInfo/Opciones de servicio/3/Retiro en tienda,additionalInfo/Opciones de servicio/3/Servicios en el lugar,additionalInfo/Opciones de servicio/4/Compras en tienda,additionalInfo/Opciones de servicio/4/Consumo en el lugar,additionalInfo/Opciones de servicio/4/Entrega el mismo día,additionalInfo/Opciones de servicio/4/Para llevar,additionalInfo/Opciones de servicio/5/Consumo en el lugar,additionalInfo/Opciones de servicio/5/Servicios en el lugar,additionalInfo/Opciones de servicio/6/Entrega el mismo día,additionalInfo/Opciones del local/0/Cena,additionalInfo/Opciones del local/0/Espacio con asientos,additionalInfo/Opciones del local/1/Servicio en el mostrador,additionalInfo/Opciones del local/2/Postres,additionalInfo/Opciones del local/3/Espacio con asientos,additionalInfo/Pagos/0/Cheques,additionalInfo/Pagos/0/Pagos móviles mediante NFC,additionalInfo/Pagos/0/Tarjetas de crédito,additionalInfo/Pagos/0/Tarjetas de débito,additionalInfo/Pagos/1/Tarjetas de crédito,additionalInfo/Pagos/1/Tarjetas de débito,additionalInfo/Pagos/2/Tarjetas de crédito,additionalInfo/Pagos/2/Tarjetas de débito,additionalInfo/Pagos/3/Tarjetas de crédito,additionalInfo/Planificación/0/Se aceptan reservas,additionalInfo/Planificación/0/Se requiere una cita,additionalInfo/Planificación/0/Visita rápida,additionalInfo/Planificación/1/Visita rápida,additionalInfo/Popular por/0/Cena,additionalInfo/Popular por/1/Cena en solitario,additionalInfo/Público usual/0/Amigable con LGBTQ+,additionalInfo/Público usual/0/Grupos,additionalInfo/Público usual/1/Espacio seguro para personas transgénero,additionalInfo/Público usual/1/Recomendable para ir en familia,additionalInfo/Público usual/2/Turistas,additionalInfo/Qué ofrece/0/Alco

In [90]:
df_tandil_05.shape

(43, 244)

In [91]:
df_tandil_05 = df_tandil_05[[
        'title',
        'categoryName',
        'website',
        'street',
        'phone',
        'city',
        'state',
        'location/lat', 
        'location/lng',
        'totalScore',
        'rank',
        'imageUrl',
        'imagesCount',       
        'reviewsCount', 
        'scrapedAt',
        'searchString']]
df_tandil_05.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Guada Vig Ventas,Tienda de ropa,NaN,NaN,+54 249 437-4284,NaN,NaN,-37.335785,-59.195321,NaN,9,NaN,NaN,0,2025-05-28T23:43:07.377Z,ventas
1,Noelia Cuesta propiedades,Agencia inmobiliaria,NaN,Sarmiento 501,+54 9 249 458-2489,Tandil,Provincia de Buenos Aires,-37.325706,-59.138529,5.0,8,https://streetviewpixels-pa.googleapis.com/v1/...,4.0,1,2025-05-28T23:43:07.377Z,ventas
2,Terrenos en venta,Urbanización,NaN,Payró 900,NaN,Tandil,Provincia de Buenos Aires,-37.327682,-59.151759,NaN,7,https://lh3.googleusercontent.com/gps-cs-s/AC9...,10.0,0,2025-05-28T23:43:07.377Z,ventas
3,Aumentar Ventas,Agencia de marketing,NaN,Muñiz 536,+54 249 424-7264,Tandil,Provincia de Buenos Aires,-37.310531,-59.102045,NaN,6,https://streetviewpixels-pa.googleapis.com/v1/...,3.0,0,2025-05-28T23:43:07.377Z,ventas
4,LAS DINAS Venta al público,Tienda de alimentación,NaN,Francia,NaN,Buenos Aires,Provincia de Buenos Aires,-37.367534,-59.132413,4.5,4,https://lh3.googleusercontent.com/gps-cs-s/AC9...,279.0,488,2025-05-28T23:43:07.376Z,ventas


In [92]:
df_tandil_05.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         43 non-null     object 
 1   categoryName  41 non-null     object 
 2   website       21 non-null     object 
 3   street        41 non-null     object 
 4   phone         34 non-null     object 
 5   city          42 non-null     object 
 6   state         42 non-null     object 
 7   location/lat  43 non-null     float64
 8   location/lng  43 non-null     float64
 9   totalScore    34 non-null     float64
 10  rank          43 non-null     int64  
 11  imageUrl      42 non-null     object 
 12  imagesCount   42 non-null     float64
 13  reviewsCount  43 non-null     int64  
 14  scrapedAt     43 non-null     object 
 15  searchString  43 non-null     object 
dtypes: float64(4), int64(2), object(10)
memory usage: 5.5+ KB


In [93]:
df_tandil_05.to_csv('../data/processed/01_dataPrep/tandil/05_tandil.csv', index=False)

In [94]:
df_tandil_06 = pd.read_csv('../data/raw/tandil_dataset_crawler-google-places_2025-05-29_00-18-14-608.csv')
df_tandil_06.head(5)

,additionalInfo/Accesibilidad/0/Entrada accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Bucle magnético de asistencia,additionalInfo/Accesibilidad/1/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/2/Bucle magnético de asistencia,additionalInfo/Accesibilidad/2/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/2/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/3/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/4/Bucle magnético de asistencia,additionalInfo/Ambiente/0/A la moda,additionalInfo/Ambiente/1/Agradable,additionalInfo/Ambiente/2/Informal,additionalInfo/Ambiente/3/Relajado,additionalInfo/Amenities/0/Desayuno incluido,additionalInfo/Amenities/0/Wi-Fi gratis,additionalInfo/Amenities/1/Desayuno,additionalInfo/Amenities/2/Estacionamiento gratuito,additionalInfo/Amenities/3/Accesible,additionalInfo/Amenities/4/Piscina cubierta / aire libre,additionalInfo/Amenities/5/Aire acondicionado,additionalInfo/Amenities/6/Centro de negocios,additionalInfo/Amenities/7/Se permiten mascotas,additionalInfo/Amenities/8/Servicio en la habitación,additionalInfo/Amenities/9/Apto para niños,additionalInfo/Amenities/10/Restaurante,additionalInfo/Amenities/11/Transporte desde/hacia el aeropuerto,additionalInfo/Amenities/12/Jacuzzi,additionalInfo/Amenities/13/Spa,additionalInfo/Amenities/14/Gimnasio,additionalInfo/Amenities/15/Campo de golf,additionalInfo/Amenities/16/Bar,additionalInfo/Amenities/17/Libre de humo,additionalInfo/Aspectos destacados/0/Buenos productos frescos,additionalInfo/Estacionamiento/0/Estacionamiento en el lugar,additionalInfo/Estacionamiento/0/Estacionamiento gratuito,additionalInfo/Estacionamiento/0/Estacionamiento gratuito en la calle,additionalInfo/Estacionamiento/1/Estacionamiento gratuito,additionalInfo/Estacionamiento/1/Estacionamiento gratuito en la calle,additionalInfo/Estacionamiento/2/Estacionamiento gratuito en la calle,additionalInfo/Estacionamiento/2/Garage de stacionamiento gratuito,additionalInfo/Estacionamiento/3/Garage de stacionamiento gratuito,additionalInfo/Información proporcionada por la empresa/0/Se identifica como mujer empresaria,additionalInfo/Menores/0/Ideal para ir con niños,additionalInfo/Opciones de servicio/0/Citas en línea,additionalInfo/Opciones de servicio/0/Clases en línea,additionalInfo/Opciones de servicio/0/Entrega a domicilio,additionalInfo/Opciones de servicio/0/Entrega el mismo día,additionalInfo/Opciones de servicio/0/Entrega sin contacto,additionalInfo/Opciones de servicio/0/Estimaciones en línea,additionalInfo/Opciones de servicio/0/Retiro en tienda,additionalInfo/Opciones de servicio/0/Retiros en la puerta,additionalInfo/Opciones de servicio/0/Servicios en el lugar,additionalInfo/Opciones de servicio/1/Citas en línea,additionalInfo/Opciones de servicio/1/Compras en tienda,additionalInfo/Opciones de servicio/1/Entrega a domicilio,additionalInfo/Opciones de servicio/1/Entrega el mismo día,additionalInfo/Opciones de servicio/1/Entrega sin contacto,additionalInfo/Opciones de servicio/1/Estimaciones en línea,additionalInfo/Opciones de servicio/1/Retiro en tienda,additionalInfo/Opciones de servicio/1/Servicios en el lugar,additionalInfo/Opciones de servicio/2/Compras en tienda,additionalInfo/Opciones de servicio/2/Entrega a domicilio,additionalInfo/Opciones de servicio/2/Entrega el mismo día,additionalInfo/Opciones de servicio/2/Para llevar,additionalInfo/Opciones de servicio/2/Retiro en tienda,additionalInfo/Opciones de servicio/2/Servicios en el lugar,additionalInfo/Opciones de ser

In [95]:
df_tandil_06.shape

(854, 480)

In [96]:
df_tandil_06 = df_tandil_06[[
        'title',
        'categoryName',
        'website',
        'street',
        'phone',
        'city',
        'state',
        'location/lat', 
        'location/lng',
        'totalScore',
        'rank',
        'imageUrl',
        'imagesCount',       
        'reviewsCount', 
        'scrapedAt',
        'searchString']]
df_tandil_06.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Lencería Anita,Tienda de lencería,NaN,Av. Colón 1545,+54 249 464-3593,AZG,Provincia de Buenos Aires,-37.312679,-59.137278,NaN,49,https://lh3.googleusercontent.com/gps-proxy/AL...,2.0,0,2025-05-29T00:13:30.912Z,ropa
1,"OLAFIL Tandil. Todo en filtros, lubricantes y ...",Tienda de repuestos para automóviles,https://www.olafil.com.ar/,Machado 504,+54 249 449-9153,Tandil,Provincia de Buenos Aires,-37.314749,-59.144955,4.6,20,https://lh3.googleusercontent.com/p/AF1QipOYYl...,19.0,28,2025-05-29T00:13:30.449Z,online
2,La Escoba Loca,Tienda de artículos para el hogar,NaN,Av. Colón 1233,NaN,Tandil,Provincia de Buenos Aires,-37.316444,-59.135010,3.8,47,https://lh3.googleusercontent.com/gps-proxy/AL...,12.0,4,2025-05-29T00:13:30.912Z,ropa
3,Encantados,Tienda de ropa para bebés,NaN,Leandro Alem 1210,+54 249 431-3563,Gran Buenos Aires,Provincia de Buenos Aires,-37.317960,-59.138314,NaN,46,https://lh3.googleusercontent.com/gps-proxy/AL...,2.0,0,2025-05-29T00:13:30.912Z,ropa
4,Mi Dulce Bebé,Tienda de ropa para bebés,NaN,Gral. Rodríguez 1181,+54 249 442-5702,Gran Buenos Aires,Provincia de Buenos Aires,-37.319635,-59.140473,NaN,45,https://lh3.googleusercontent.com/gps-cs-s/AC9...,5.0,0,2025-05-29T00:13:30.912Z,ropa


In [97]:
df_tandil_06.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 854 entries, 0 to 853
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         854 non-null    object 
 1   categoryName  838 non-null    object 
 2   website       315 non-null    object 
 3   street        830 non-null    object 
 4   phone         697 non-null    object 
 5   city          844 non-null    object 
 6   state         845 non-null    object 
 7   location/lat  854 non-null    float64
 8   location/lng  854 non-null    float64
 9   totalScore    692 non-null    float64
 10  rank          854 non-null    int64  
 11  imageUrl      827 non-null    object 
 12  imagesCount   827 non-null    float64
 13  reviewsCount  854 non-null    int64  
 14  scrapedAt     854 non-null    object 
 15  searchString  854 non-null    object 
dtypes: float64(4), int64(2), object(10)
memory usage: 106.9+ KB


In [98]:
df_tandil_06.to_csv('../data/processed/01_dataPrep/tandil/06_tandil.csv', index=False)

In [99]:
df_tandil_07 = pd.read_csv('../data/raw/tandil_dataset_crawler-google-places_2025-06-11_18-03-13-154.csv')
df_tandil_07.head(5)

,additionalInfo/Accesibilidad/0/Entrada accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Bucle magnético de asistencia,additionalInfo/Accesibilidad/1/Entrada accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/2/Bucle magnético de asistencia,additionalInfo/Accesibilidad/2/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/2/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/2/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/3/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Ambiente/0/A la moda,additionalInfo/Ambiente/0/Agradable,additionalInfo/Ambiente/1/Agradable,additionalInfo/Ambiente/1/Informal,additionalInfo/Ambiente/2/Histórico,additionalInfo/Ambiente/2/Informal,additionalInfo/Ambiente/2/Relajado,additionalInfo/Ambiente/3/Informal,additionalInfo/Ambiente/3/Relajado,additionalInfo/Ambiente/4/Relajado,additionalInfo/Amenities/0/Wi-Fi gratis,additionalInfo/Aspectos destacados/0/Asientos en la terraza,additionalInfo/Aspectos destacados/0/Buen café,additionalInfo/Aspectos destacados/0/Buenos productos frescos,additionalInfo/Aspectos destacados/0/Deliciosos postres,additionalInfo/Aspectos destacados/0/Muy buena selección de cervezas,additionalInfo/Aspectos destacados/1/Buen café,additionalInfo/Aspectos destacados/1/Deliciosos postres,additionalInfo/Aspectos destacados/1/Muy buena selección de cervezas,additionalInfo/Aspectos destacados/2/Buenos cócteles,additionalInfo/Aspectos destacados/2/Chimenea,additionalInfo/Aspectos destacados/2/Deportes,additionalInfo/Aspectos destacados/2/Muy buena selección de cervezas,additionalInfo/Aspectos destacados/2/Muy buena selección de té,additionalInfo/Aspectos destacados/3/Chimenea,additionalInfo/Aspectos destacados/3/Deliciosos postres,additionalInfo/Aspectos destacados/3/Muy buena selección de cervezas,additionalInfo/Aspectos destacados/3/Muy buena selección de té,additionalInfo/Aspectos destacados/3/Muy buena selección de vinos,additionalInfo/Aspectos destacados/4/Deliciosos postres,additionalInfo/Aspectos destacados/4/Muy buena selección de cervezas,additionalInfo/Aspectos destacados/4/Muy buena selección de té,additionalInfo/Aspectos destacados/5/Deportes,additionalInfo/Aspectos destacados/6/Música en vivo,additionalInfo/Aspectos destacados/7/Muy buena selección de cervezas,additionalInfo/Aspectos destacados/8/Muy buena selección de té,additionalInfo/Estacionamiento/0/Estacionamiento en el lugar,additionalInfo/Estacionamiento/0/Estacionamiento gratuito,additionalInfo/Estacionamiento/0/Estacionamiento gratuito en la calle,additionalInfo/Estacionamiento/0/Estacionamiento pagado,additionalInfo/Estacionamiento/0/Hay suficiente espacio,additionalInfo/Estacionamiento/1/Estacionamiento gratuito,additionalInfo/Estacionamiento/1/Estacionamiento gratuito en la calle,additionalInfo/Estacionamiento/1/Estacionamiento pagado en la calle,additionalInfo/Estacionamiento/1/Garage de stacionamiento gratuito,additionalInfo/Estacionamiento/2/Estacionamiento gratuito en la calle,additionalInfo/Información proporcionada por la empresa/0/Se identifica como mujer empresaria,additionalInfo/Mascotas/0/Se admiten perros en el interior,additionalInfo/Mascotas/0/Se permiten perros,additionalInfo/Mascotas/1/Se permiten perros,additionalInfo/Mascotas/2/Se permiten perros afuera,additionalInfo/Menores/0/Ideal para ir con niños,additionalInfo/Menor

In [100]:
df_tandil_07.shape

(735, 475)

In [101]:
df_tandil_07 = df_tandil_07[[
        'title',
        'categoryName',
        'website',
        'street',
        'phone',
        'city',
        'state',
        'location/lat', 
        'location/lng',
        'totalScore',
        'rank',
        'imageUrl',
        'imagesCount',       
        'reviewsCount', 
        'scrapedAt',
        'searchString']]
df_tandil_07.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Marcovecchio Bikes,Tienda de bicicletas,https://marcovecchiobikes.com/,"Astor Piazzola 160 Entre Ijurco y, Cesar Milstein",+54 249 401-9991,Tandil,Provincia de Buenos Aires,-37.282066,-59.102040,4.9,1,https://lh3.googleusercontent.com/p/AF1QipNLaf...,76.0,371,2025-06-11T18:01:49.484Z,venta online
1,El Ultimo Recurso,Comercio,NaN,Ijurco 2243,NaN,Tandil,Provincia de Buenos Aires,-37.294901,-59.115855,5.0,1,https://lh3.googleusercontent.com/gps-cs-s/AC9...,2.0,1,2025-06-11T18:01:50.668Z,comercio
2,Enever tandil,Comercio,https://www.instagram.com/enever_tandil?igsh=a...,"pasaje interno 1850 ENTRE LA CALLE IJURCO, Juá...",+54 249 428-7471,Tandil,Provincia de Buenos Aires,-37.298936,-59.120144,5.0,2,https://lh3.googleusercontent.com/p/AF1QipP2Tl...,6.0,1,2025-06-11T18:01:50.668Z,comercio
3,El Surtidor Tandil,Comercio,https://sites.google.com/view/el-surtidor-tand...,"Darragueira, Ijurco 1999",+54 249 462-2606,Tandil,Provincia de Buenos Aires,-37.300845,-59.123275,4.4,3,https://lh3.googleusercontent.com/p/AF1QipNqjA...,18.0,280,2025-06-11T18:01:50.668Z,comercio
4,A limpiar,Comercio,NaN,Ijurco 1883,+54 249 464-4277,Tandil,Provincia de Buenos Aires,-37.298287,-59.120049,4.3,4,https://lh3.googleusercontent.com/p/AF1QipMcYB...,39.0,47,2025-06-11T18:01:50.668Z,comercio


In [102]:
df_tandil_07.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         735 non-null    object 
 1   categoryName  721 non-null    object 
 2   website       349 non-null    object 
 3   street        711 non-null    object 
 4   phone         596 non-null    object 
 5   city          731 non-null    object 
 6   state         732 non-null    object 
 7   location/lat  735 non-null    float64
 8   location/lng  735 non-null    float64
 9   totalScore    658 non-null    float64
 10  rank          735 non-null    int64  
 11  imageUrl      726 non-null    object 
 12  imagesCount   726 non-null    float64
 13  reviewsCount  735 non-null    int64  
 14  scrapedAt     735 non-null    object 
 15  searchString  735 non-null    object 
dtypes: float64(4), int64(2), object(10)
memory usage: 92.0+ KB


In [103]:
df_tandil_07.to_csv('../data/processed/01_dataPrep/tandil/07_tandil.csv', index=False)

In [104]:
df_tandil_08 = pd.read_csv('../data/raw/tandil_dataset_crawler-google-places_2025-06-15_23-39-41-132.csv')
df_tandil_08.head(5)

,additionalInfo/Accesibilidad/0/Entrada accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/0/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Espacio accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/1/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Accesibilidad/2/Estacionamiento accesible para personas en silla de ruedas,additionalInfo/Accesibilidad/3/Sanitarios accesibles para personas en silla de ruedas,additionalInfo/Ambiente/0/A la moda,additionalInfo/Ambiente/1/Agradable,additionalInfo/Ambiente/2/Informal,additionalInfo/Ambiente/3/Relajado,additionalInfo/Aspectos destacados/0/Buenos productos frescos,additionalInfo/Estacionamiento/0/Estacionamiento en el lugar,additionalInfo/Estacionamiento/0/Estacionamiento gratuito,additionalInfo/Estacionamiento/0/Estacionamiento gratuito en la calle,additionalInfo/Estacionamiento/1/Estacionamiento gratuito,additionalInfo/Estacionamiento/1/Estacionamiento gratuito en la calle,additionalInfo/Estacionamiento/2/Estacionamiento gratuito en la calle,additionalInfo/Estacionamiento/2/Garage de stacionamiento gratuito,additionalInfo/Información proporcionada por la empresa/0/Se identifica como mujer empresaria,additionalInfo/Mascotas/0/Se permiten perros,additionalInfo/Menores/0/Ideal para ir con niños,additionalInfo/Opciones de servicio/0/Citas en línea,additionalInfo/Opciones de servicio/0/Entrega a domicilio,additionalInfo/Opciones de servicio/0/Entrega el mismo día,additionalInfo/Opciones de servicio/0/Retiro en tienda,additionalInfo/Opciones de servicio/0/Retiros en la puerta,additionalInfo/Opciones de servicio/0/Servicios en el lugar,additionalInfo/Opciones de servicio/1/Citas en línea,additionalInfo/Opciones de servicio/1/Compras en tienda,additionalInfo/Opciones de servicio/1/Entrega a domicilio,additionalInfo/Opciones de servicio/1/Entrega el mismo día,additionalInfo/Opciones de servicio/1/Para llevar,additionalInfo/Opciones de servicio/1/Retiro en tienda,additionalInfo/Opciones de servicio/2/Compras en tienda,additionalInfo/Opciones de servicio/2/Entrega a domicilio,additionalInfo/Opciones de servicio/2/Entrega el mismo día,additionalInfo/Opciones de servicio/2/Retiro en tienda,additionalInfo/Opciones de servicio/2/Servicios en el lugar,additionalInfo/Opciones de servicio/3/Citas en línea,additionalInfo/Opciones de servicio/3/Compras en tienda,additionalInfo/Opciones de servicio/3/Entrega el mismo día,additionalInfo/Opciones de servicio/3/Retiro en tienda,additionalInfo/Opciones de servicio/4/Compras en tienda,additionalInfo/Opciones de servicio/4/Entrega a domicilio,additionalInfo/Opciones de servicio/4/Entrega el mismo día,additionalInfo/Opciones de servicio/4/Servicios en el lugar,additionalInfo/Opciones de servicio/5/Entrega el mismo día,additionalInfo/Opciones de servicio/5/Servicios en el lugar,additionalInfo/Opciones de servicio/6/Entrega el mismo día,additionalInfo/Opciones del local/0/Desayunos,additionalInfo/Pagos/0/Cheques,additionalInfo/Pagos/0/Pagos móviles mediante NFC,additionalInfo/Pagos/0/Solo dinero en efectivo,additionalInfo/Pagos/0/Tarjetas de crédito,additionalInfo/Pagos/0/Tarjetas de débito,additionalInfo/Pagos/1/Pagos móviles mediante NFC,additionalInfo/Pagos/1/Solo dinero en efectivo,additionalInfo/Pagos/1/Tarjetas de crédito,additionalInfo/Pagos/1/Tarjetas de débito,additionalInfo/Pagos/2/Solo dinero en efectivo,additionalInfo/Pagos/2/Tarjetas de crédito,additionalInfo/Pagos/2/Tarjetas de débito,additionalInfo/Pagos/3/Tarjetas de crédito,additionalInfo/Pagos/3/Tarjetas de débito,additionalInfo/Pagos/4/Tarjetas de crédito,additionalInfo/Pagos/4/Tarjetas de débito,additionalInfo/Pagos/5/Tarjetas de créd

In [105]:
df_tandil_08.shape

(207, 414)

In [106]:
df_tandil_08 = df_tandil_08[[
        'title',
        'categoryName',
        'website',
        'street',
        'phone',
        'city',
        'state',
        'location/lat', 
        'location/lng',
        'totalScore',
        'rank',
        'imageUrl',
        'imagesCount',       
        'reviewsCount', 
        'scrapedAt',
        'searchString']]
df_tandil_08.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Springwall | Tandil,Colchonería,https://www.springwall.com.ar/,Av. Avellaneda 1310,+54 9 249 468-8170,Tandil,Provincia de Buenos Aires,-37.329427,-59.121875,4.9,19,https://lh3.googleusercontent.com/p/AF1QipMWiH...,6.0,35,2025-06-15T23:38:00.068Z,blanqueria
1,Hanna Boutique,Tienda de ropa,NaN,9 de Julio 246,NaN,Tandil,Provincia de Buenos Aires,-37.330272,-59.132935,4.8,13,https://lh3.googleusercontent.com/gps-cs-s/AC9...,2.0,6,2025-06-15T23:38:00.068Z,blanqueria
2,DecoRA Tandil,Tienda de cortinas,https://sites.google.com/view/decoratandil,Mitre 855,+54 249 459-7566,Tandil,Provincia de Buenos Aires,-37.322707,-59.134192,4.1,11,https://lh3.googleusercontent.com/p/AF1QipN6S7...,29.0,12,2025-06-15T23:38:00.067Z,blanqueria
3,Nube,Tienda de ropa para bebés,NaN,Quintana 565,+54 249 442-4159,Tandil,Provincia de Buenos Aires,-37.307663,-59.142191,3.8,10,https://lh3.googleusercontent.com/gps-proxy/AL...,2.0,4,2025-06-15T23:38:00.067Z,blanqueria
4,Sweet Dreams Colchoneria y Blanqueria,Colchonería,NaN,Honduras 1182,+54 249 461-6601,Tandil,Provincia de Buenos Aires,-37.307517,-59.125232,5.0,9,https://lh3.googleusercontent.com/p/AF1QipMGuA...,19.0,7,2025-06-15T23:38:00.067Z,blanqueria


In [107]:
df_tandil_08.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         207 non-null    object 
 1   categoryName  199 non-null    object 
 2   website       82 non-null     object 
 3   street        196 non-null    object 
 4   phone         161 non-null    object 
 5   city          205 non-null    object 
 6   state         206 non-null    object 
 7   location/lat  207 non-null    float64
 8   location/lng  207 non-null    float64
 9   totalScore    179 non-null    float64
 10  rank          207 non-null    int64  
 11  imageUrl      201 non-null    object 
 12  imagesCount   201 non-null    float64
 13  reviewsCount  207 non-null    int64  
 14  scrapedAt     207 non-null    object 
 15  searchString  207 non-null    object 
dtypes: float64(4), int64(2), object(10)
memory usage: 26.0+ KB


In [108]:
df_tandil_08.to_csv('../data/processed/01_dataPrep/tandil/08_tandil.csv', index=False)

In [109]:
glob.glob('../data/processed/01_dataPrep/tandil/*.csv')

['../data/processed/01_dataPrep/tandil/05_tandil.csv',
 '../data/processed/01_dataPrep/tandil/02_tandil.csv',
 '../data/processed/01_dataPrep/tandil/03_tandil.csv',
 '../data/processed/01_dataPrep/tandil/08_tandil.csv',
 '../data/processed/01_dataPrep/tandil/04_tandil.csv',
 '../data/processed/01_dataPrep/tandil/01_tandil.csv',
 '../data/processed/01_dataPrep/tandil/07_tandil.csv',
 '../data/processed/01_dataPrep/tandil/06_tandil.csv']

In [110]:
df = pd.concat(map(pd.read_csv, glob.glob('../data/processed/01_dataPrep/tandil/*.csv')))
df.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Guada Vig Ventas,Tienda de ropa,NaN,NaN,+54 249 437-4284,NaN,NaN,-37.335785,-59.195321,NaN,9,NaN,NaN,0,2025-05-28T23:43:07.377Z,ventas
1,Noelia Cuesta propiedades,Agencia inmobiliaria,NaN,Sarmiento 501,+54 9 249 458-2489,Tandil,Provincia de Buenos Aires,-37.325706,-59.138529,5.0,8,https://streetviewpixels-pa.googleapis.com/v1/...,4.0,1,2025-05-28T23:43:07.377Z,ventas
2,Terrenos en venta,Urbanización,NaN,Payró 900,NaN,Tandil,Provincia de Buenos Aires,-37.327682,-59.151759,NaN,7,https://lh3.googleusercontent.com/gps-cs-s/AC9...,10.0,0,2025-05-28T23:43:07.377Z,ventas
3,Aumentar Ventas,Agencia de marketing,NaN,Muñiz 536,+54 249 424-7264,Tandil,Provincia de Buenos Aires,-37.310531,-59.102045,NaN,6,https://streetviewpixels-pa.googleapis.com/v1/...,3.0,0,2025-05-28T23:43:07.377Z,ventas
4,LAS DINAS Venta al público,Tienda de alimentación,NaN,Francia,NaN,Buenos Aires,Provincia de Buenos Aires,-37.367534,-59.132413,4.5,4,https://lh3.googleusercontent.com/gps-cs-s/AC9...,279.0,488,2025-05-28T23:43:07.376Z,ventas


In [111]:
df.to_csv('../data/processed/01_dataPrep/main/main_tandil.csv', index=False)

# Data Cleaning

In [112]:
# Se carga el archivo .CSV principal de la ciudad
df_main_azul = pd.read_csv('../data/processed/01_dataPrep/main/main_azul.csv')

In [113]:
# Se ordena el dataframe en orden alfabético y se vizualizan las primeras 5 filas
df_main_azul = df_main_azul.sort_values(by='title', ascending=True)
df_main_azul.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
336,"""Me importa un comino""",Tienda de alimentos naturales,NaN,Olavarría 1249,+54 2281 53-8734,Azul,Provincia de Buenos Aires,-36.791438,-59.854428,4.5,49,https://lh3.googleusercontent.com/p/AF1QipNBGn...,1.0,2,2025-05-19T00:03:04.970Z,Comercio
165,190 grados hierro&madera,Fábrica de muebles,NaN,Lavalle 1372,+54 2281 41-0748,Azul,Provincia de Buenos Aires,-36.794246,-59.867392,5.0,16,https://lh3.googleusercontent.com/p/AF1QipMDmE...,14.0,2,2025-06-15T22:57:48.198Z,muebles
141,AGRICOLA AZUL SA,Proveedor de máquinas de metal,NaN,"RN3 km 300,5",+54 2281 58-4563,Azul,Provincia de Buenos Aires,-36.802081,-59.852856,4.8,7,https://lh3.googleusercontent.com/gps-cs-s/AC9...,29.0,17,2025-06-15T22:57:22.428Z,maquinaria
60,AGROREPUESTOS AZUL,Servicio agrícola,NaN,RN3 km 301,+54 2281 65-4471,Azul,Provincia de Buenos Aires,-36.804447,-59.855641,4.6,55,https://lh3.googleusercontent.com/p/AF1QipMz4V...,16.0,5,2025-06-15T22:57:08.021Z,repuestos
47,ALMÍBAR,Tienda de alimentación,NaN,"Calle Gral. Paz y Comercio, B7300EVX",+54 2281 50-9728,Azul,Provincia de Buenos Aires,-36.795313,-59.857420,4.7,3,https://lh3.googleusercontent.com/gps-cs-s/AC9...,16.0,6,2025-06-15T22:56:57.476Z,comercio


In [114]:
# Este código cuenta la frecuencia de cada registro en la columna 'state'
df_main_azul['state'].value_counts()

state
Provincia de Buenos Aires    516
Name: count, dtype: int64

In [115]:
# Se cuenta la frecuencia de los registros de la columna 'city'
df_main_azul['city'].value_counts()

city
Azul            503
Buenos Aires      4
B7300GEU          2
CJV               2
FAD               1
FRU               1
GEV               1
CJU               1
CKA               1
Name: count, dtype: int64

In [116]:
# Se reemplazan los datos de la columna 'city' que no son correctos
df_main_azul['city'] = df_main_azul['city'].replace({
    'Buenos Aires': 'Azul',
    'FRU' : 'Azul',
    'FAD' : 'Azul',
    'B7300GEU' : 'Azul',
    'CJV' : 'Azul',
    'GEV' : 'Azul',
    'CJU' : 'Azul',
    'CKA' : 'Azul'
})

In [117]:
# Verificacion de datos reemplazados
df_main_azul['city'].value_counts()

city
Azul    516
Name: count, dtype: int64

In [118]:
# Se guarda el dataframe en archivo .CSV
df_main_azul.to_csv('../data/processed/02_city_Data/azul_Data.csv', index=False)

In [119]:
# Se lee archivo .CSV
df_main_juarez = pd.read_csv('../data/processed/01_dataPrep/main/main_juarez.csv')

In [120]:
# Visualizacion primeras 5 filas
df_main_juarez.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Serantes Accesorios,Tienda de repuestos de automóviles usados,NaN,Av. Muñiz 49,+54 2281 46-6108,Benito Juárez,Provincia de Buenos Aires,-37.678436,-59.800652,4.6,1,https://lh3.googleusercontent.com/gps-cs-s/AC9...,2.0,66,2025-05-29T00:53:21.747Z,repuestos
1,Carlos REYNOSO REPUESTOS AGRICOLAS,Tienda de repuestos para coches de carreras,NaN,Av. Muñiz 251-299,NaN,Benito Juárez,Provincia de Buenos Aires,-37.682303,-59.805742,4.9,2,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,11,2025-05-29T00:53:21.748Z,repuestos
2,Repuestos lo de migue,Tienda de accesorios para automóviles,NaN,San Juan 126,+54 2281 53-2680,Benito Juárez,Provincia de Buenos Aires,-37.672441,-59.800303,4.6,3,https://lh3.googleusercontent.com/gps-cs-s/AC9...,4.0,17,2025-05-29T00:53:21.748Z,repuestos
3,Repuestos Perco,Tienda de repuestos de automóviles usados,NaN,Av. Urquiza 41,+54 2281 53-2090,Benito Juárez,Provincia de Buenos Aires,-37.672897,-59.803693,4.6,4,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,5,2025-05-29T00:53:21.749Z,repuestos
4,Acoplados Juarez - Repuestos acoplados y semir...,Proveedor de repuestos de carrocería de automó...,https://www.guia-dorada.com.ar/guia/aviso-repu...,"Ruta Provincial 86, Suipacha y",+54 9 249 463-4600,Benito Juárez,Provincia de Buenos Aires,-37.679447,-59.818685,5.0,5,https://lh3.googleusercontent.com/gps-cs-s/AC9...,26.0,2,2025-05-29T00:53:21.749Z,repuestos


In [121]:
# Imprime cuántas veces aparece cada categoría en 'state'
df_main_juarez['state'].value_counts()

state
Provincia de Buenos Aires    385
Name: count, dtype: int64

In [122]:
# Imprime cuántas veces aparece cada categoría en 'city', ordenadas de mayor a menor
df_main_juarez['city'].value_counts()

city
Benito Juárez    378
Buenos Aires       3
Henderson          2
alberti            1
BENITO JUAREZ      1
Name: count, dtype: int64

In [123]:
# Reemplazo de variantes de ciudad por 'Benito Juárez' y muestra su frecuencia tras estandarizar
df_main_juarez['city'] = df_main_juarez['city'].replace({
    'Buenos Aires': 'Benito Juárez',
    'Henderson' : 'Benito Juárez',
    'BENITO JUAREZ' : 'Benito Juárez',
    'alberti' : 'Benito Juárez'
})
df_main_juarez['city'].value_counts()

city
Benito Juárez    385
Name: count, dtype: int64

In [124]:
# Se guarda en archivo .CSV
df_main_juarez.to_csv('../data/processed/02_city_Data/juarez_Data.csv', index=False)

In [125]:
# Lectura de archivo .CSV
df_main_olavarria = pd.read_csv('../data/processed/01_dataPrep/main/main_olavarria.csv')
df_main_olavarria.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Polirrubros La Silvia,Kiosco,NaN,Av. Ituzaingó 1317,NaN,Olavarría,Provincia de Buenos Aires,-36.895466,-60.345100,4.8,1,https://lh3.googleusercontent.com/gps-cs-s/AC9...,16.0,27,2025-05-29T00:56:12.806Z,comercio
1,Distribuidora Katy,Comercio,NaN,"Pelegrino, Av. Ituzaingó e",+54 2284 71-1461,Olavarría,Provincia de Buenos Aires,-36.890448,-60.338133,NaN,2,https://lh3.googleusercontent.com/p/AF1QipOckQ...,20.0,0,2025-05-29T00:56:12.808Z,comercio
2,Mercadito Gaby-Fer,Comercio,NaN,Collinet 2595,+54 2284 41-7540,Olavarría,Provincia de Buenos Aires,-36.900998,-60.335121,5.0,3,https://lh3.googleusercontent.com/gps-cs-s/AC9...,3.0,4,2025-05-29T00:56:12.808Z,comercio
3,Dulcemente,Tienda de alimentación,NaN,Independencia 1700-1600,+54 2284 69-7125,Olavarría,Provincia de Buenos Aires,-36.895493,-60.339700,5.0,4,https://lh3.googleusercontent.com/gps-cs-s/AC9...,2.0,2,2025-05-29T00:56:12.808Z,comercio
4,104 Polirrubro,Tienda de alimentación,NaN,Azopardo 1598,+54 2284 42-1670,Olavarría,Provincia de Buenos Aires,-36.893022,-60.341844,3.8,5,https://lh3.googleusercontent.com/gps-cs-s/AC9...,18.0,4,2025-05-29T00:56:12.808Z,comercio


In [126]:
# Se cuenta la frecuencia de cada registro en la columna 'state'
df_main_olavarria['state'].value_counts()

state
Provincia de Buenos Aires          1088
CABA                                  1
Ciudad Autónoma de Buenos Aires       1
Name: count, dtype: int64

In [127]:
# Reemplazo de valores erróneos
df_main_olavarria['state'] = df_main_olavarria['state'].replace({
    'CABA' : 'Provincia de Buenos Aires',
    'Ciudad Autónoma de Buenos Aires' : 'Provincia de Buenos Aires'
})

In [128]:
# Imprime cuántas veces aparece cada categoría en 'city'
df_main_olavarria['city'].value_counts()

city
Olavarría            1027
Gran Buenos Aires      32
IDH                     3
DFE                     3
KKJ                     3
Buenos Aires            2
COO                     2
B7400LCX                2
CZY                     2
CVQ                     2
KCI                     2
B7400JWP                1
CUV                     1
CABA                    1
Moreno                  1
Caseros                 1
JZB                     1
DAS                     1
Colon                   1
LLD                     1
DJY                     1
Name: count, dtype: int64

In [129]:
# Reemplazo de variantes de ciudad por 'Olavarría'
df_main_olavarria['city'] = df_main_olavarria['city'].replace({
    'Buenos Aires': 'Olavarría',
    'Gran Buenos Aires' : 'Olavarría',
    'Colon' : 'Olavarría',
    'B7400JWP' : 'Olavarría',
    'CZY' : 'Olavarría',
    'B7400LCX' : 'Olavarría',
    'KCI' : 'Olavarría',
    'DAS' : 'Olavarría',
    'COO' : 'Olavarría',
    'CVQ' : 'Olavarría',
    'DFE' : 'Olavarría',
    'Caseros' : 'Olavarría',
    'IDH' : 'Olavarría',
    'JZB' : 'Olavarría',
    'CABA' : 'Olavarría',
    'KKJ' : 'Olavarría',
    'Moreno' : 'Olavarría',
    'Buenos Aires' : 'Olavarría',
    'CUV' : 'Olavarría',
    'LLD' : 'Olavarría',
    'DJY' : 'Olavarría'
})

In [130]:
# Conteos de combinaciones únicas para las columnas 'state' y 'city'
df_main_olavarria[['state', 'city']].value_counts()

state                      city     
Provincia de Buenos Aires  Olavarría    1090
Name: count, dtype: int64

In [131]:
# Guarda el DataFrame como CSV sin índice
df_main_olavarria.to_csv('../data/processed/02_city_Data/olavarria_Data.csv', index=False)

In [132]:
df_main_rauch = pd.read_csv('../data/processed/01_dataPrep/main/main_rauch.csv')
df_main_rauch.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Del Valle inyeccion electronica,Taller de reparación de automóviles,NaN,"Aristóbulo del Valle, Aveleyra &",+54 249 434-0861,Rauch,Provincia de Buenos Aires,-36.773298,-59.091459,5.0,1,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,1,2025-05-29T00:48:32.392Z,electronica
1,Electro 440 - Accesorios y Reparación,Tienda de electrónica,https://www.facebook.com/todoenremotos,Letamendi 596,+54 9 249 427-7096,Rauch,Provincia de Buenos Aires,-36.776094,-59.088072,5.0,2,https://lh3.googleusercontent.com/gps-cs-s/AC9...,35.0,5,2025-05-29T00:48:32.394Z,electronica
2,Dangerous Sonido e Iluminación,Tienda de equipos de sonido,NaN,"BNK, Saavedra 507",+54 249 453-3029,Rauch,Provincia de Buenos Aires,-36.772975,-59.093649,5.0,3,https://lh3.googleusercontent.com/gps-cs-s/AC9...,28.0,3,2025-05-29T00:48:32.394Z,electronica
3,Jona PC,Servicio de informática,https://www.facebook.com/profile.php?id=100069...,Dardo Rocha 850,NaN,Rauch,Provincia de Buenos Aires,-36.761649,-59.092823,5.0,4,https://lh3.googleusercontent.com/p/AF1QipPcWb...,13.0,11,2025-05-29T00:48:32.394Z,electronica
4,Super PC Rauch,Tienda de informática,NaN,Balcarce 1001,+54 249 435-8292,Rauch,Provincia de Buenos Aires,-36.768249,-59.090148,NaN,5,https://streetviewpixels-pa.googleapis.com/v1/...,1.0,0,2025-05-29T00:48:32.395Z,electronica


In [133]:
df_main_rauch['state'].value_counts()

state
Provincia de Buenos Aires    510
Buenos Aires Sur               3
Name: count, dtype: int64

In [134]:
df_main_rauch['state'] = df_main_rauch['state'].replace({
    'Buenos Aires Sur' : 'Provincia de Buenos Aires'
    })


In [135]:
df_main_rauch['city'].value_counts()

city
Rauch           445
BPQ              17
Buenos Aires      5
ADB               4
BPR               3
AEX               3
BBP               3
ASJ               3
ACF               3
ACD               3
AQH               3
BPJ               2
AQJ               2
ABD               2
ATQ               2
BPO               2
AKG               2
BQC               2
BID               2
BPI               1
BPP               1
BPK               1
BCB               1
AQE               1
Name: count, dtype: int64

In [136]:
df_main_rauch['city'] = df_main_rauch['city'].replace({
    'BPQ': 'Rauch',
    'BID' : 'Rauch',
    'ADB' : 'Rauch',
    'Buenos Aires' : 'Rauch',
    'BQC' : 'Rauch',
    'BCB' : 'Rauch',
    'BPK' : 'Rauch',
    'AQH' : 'Rauch',
    'ACF' : 'Rauch',
    'BPI' : 'Rauch',
    'BPP' : 'Rauch',
    'AKG' : 'Rauch',
    'ABD' : 'Rauch',
    'ACD' : 'Rauch',
    'AQJ' : 'Rauch',
    'ATQ' : 'Rauch',
    'BPO' : 'Rauch',
    'BBP' : 'Rauch',
    'AEX' : 'Rauch',
    'ASJ' : 'Rauch',
    'AQE' : 'Rauch',
    'AQE' : 'Rauch',
    'BPJ' : 'Rauch',
    'BPR' : 'Rauch'
    })
df_main_rauch[['city', 'state']].value_counts()

city   state                    
Rauch  Provincia de Buenos Aires    513
Name: count, dtype: int64

In [137]:
df_main_rauch.to_csv('../data/processed/02_city_Data/rauch_Data.csv', index=False)

In [138]:
df_main_tandil = pd.read_csv('../data/processed/01_dataPrep/main/main_tandil.csv')
df_main_tandil.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Guada Vig Ventas,Tienda de ropa,NaN,NaN,+54 249 437-4284,NaN,NaN,-37.335785,-59.195321,NaN,9,NaN,NaN,0,2025-05-28T23:43:07.377Z,ventas
1,Noelia Cuesta propiedades,Agencia inmobiliaria,NaN,Sarmiento 501,+54 9 249 458-2489,Tandil,Provincia de Buenos Aires,-37.325706,-59.138529,5.0,8,https://streetviewpixels-pa.googleapis.com/v1/...,4.0,1,2025-05-28T23:43:07.377Z,ventas
2,Terrenos en venta,Urbanización,NaN,Payró 900,NaN,Tandil,Provincia de Buenos Aires,-37.327682,-59.151759,NaN,7,https://lh3.googleusercontent.com/gps-cs-s/AC9...,10.0,0,2025-05-28T23:43:07.377Z,ventas
3,Aumentar Ventas,Agencia de marketing,NaN,Muñiz 536,+54 249 424-7264,Tandil,Provincia de Buenos Aires,-37.310531,-59.102045,NaN,6,https://streetviewpixels-pa.googleapis.com/v1/...,3.0,0,2025-05-28T23:43:07.377Z,ventas
4,LAS DINAS Venta al público,Tienda de alimentación,NaN,Francia,NaN,Buenos Aires,Provincia de Buenos Aires,-37.367534,-59.132413,4.5,4,https://lh3.googleusercontent.com/gps-cs-s/AC9...,279.0,488,2025-05-28T23:43:07.376Z,ventas


In [139]:
df_main_tandil['city'].value_counts()

city
Tandil                             2559
Gran Buenos Aires                    14
Buenos Aires                          9
HXG                                   3
BWH                                   3
IRK                                   3
FTE                                   3
BGA                                   3
ATN                                   2
GZY                                   2
GRS                                   2
GKP                                   2
AOS                                   2
IRL                                   2
AQX                                   1
Gutiérrez                             1
Ciudad Autónoma de Buenos Aires       1
EAC                                   1
BJD                                   1
AOX                                   1
BKB                                   1
AZR                                   1
FSW                                   1
AZG                                   1
AOV                                

In [140]:
df_main_tandil['city'] = df_main_tandil['city'].replace({
    'Gran Buenos Aires': 'Tandil',
    'IRL' : 'Tandil',
    'GKP' : 'Tandil',
    'Buenos Aires' : 'Tandil',
    'IRK' : 'Tandil',
    'AOS' : 'Tandil',
    'AOV' : 'Tandil',
    'GRS' : 'Tandil',
    'GQQ' : 'Tandil',
    'AOX' : 'Tandil',
    'GVL' : 'Tandil',
    'BKB' : 'Tandil',
    'AHM' : 'Tandil',
    'Ciudad Autónoma de Buenos Aires' : 'Tandil',
    'EAC' : 'Tandil',
    'AQX' : 'Tandil',
    'HXG' : 'Tandil',
    'AOP' : 'Tandil',
    'BWH' : 'Tandil',
    'AZG' : 'Tandil',
    'AOO' : 'Tandil',
    'Gutiérrez' : 'Tandil',
    'AZC' : 'Tandil',
    'AZD' : 'Tandil',
    'AZR' : 'Tandil',
    'AEK' : 'Tandil',
    'BGA' : 'Tandil',
    'FSW' : 'Tandil',
    'FTE' : 'Tandil',
    'ATN' : 'Tandil',
    'GMM' : 'Tandil',
    'BJD' : 'Tandil',
    'GKR' : 'Tandil',
    'GZY' : 'Tandil',
    })

In [141]:
df_main_tandil['state'].value_counts()

state
Provincia de Buenos Aires          2632
Ciudad Autónoma de Buenos Aires       4
de Buenos Aires                       1
Name: count, dtype: int64

In [142]:
df_main_tandil['state'] = df_main_tandil['state'].replace({
    'de Buenos Aires' : 'Provincia de Buenos Aires',
    'Ciudad Autónoma de Buenos Aires' : 'Provincia de Buenos Aires'
    })

In [143]:
df_main_rauch[['city', 'state']].value_counts()

city   state                    
Rauch  Provincia de Buenos Aires    513
Name: count, dtype: int64

In [144]:
df_main_tandil.to_csv('../data/processed/02_city_Data/tandil_Data.csv', index=False)

**

In [145]:
# Lista todos los archivos CSV en la carpeta '02_city_Data'
# usando glob con patrón '*.csv', para procesarlos automáticamente.
glob.glob('../data/processed/02_city_Data/*.csv')

['../data/processed/02_city_Data/azul_Data.csv',
 '../data/processed/02_city_Data/rauch_Data.csv',
 '../data/processed/02_city_Data/tandil_Data.csv',
 '../data/processed/02_city_Data/juarez_Data.csv',
 '../data/processed/02_city_Data/olavarria_Data.csv']

In [146]:
# Lee todos los CSV de la carpeta '02_city_Data', los combina en un solo DataFrame  y luego muestra los nombres de las columnas resultantes.
df = pd.concat(map(pd.read_csv, glob.glob('../data/processed/02_city_Data/*.csv')))
df.columns

Index(['title', 'categoryName', 'website', 'street', 'phone', 'city', 'state',
       'location/lat', 'location/lng', 'totalScore', 'rank', 'imageUrl',
       'imagesCount', 'reviewsCount', 'scrapedAt', 'searchString'],
      dtype='object')

In [147]:
# Información del DataFrame a través del método .info() : 
# Número de filas y columnas
# Nombres y tipos de datos de las columnas
# Valores no nulos en cada columna
# Uso de memoria
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5171 entries, 0 to 1092
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         5171 non-null   object 
 1   categoryName  5055 non-null   object 
 2   website       1670 non-null   object 
 3   street        4906 non-null   object 
 4   phone         3724 non-null   object 
 5   city          5134 non-null   object 
 6   state         5141 non-null   object 
 7   location/lat  5171 non-null   float64
 8   location/lng  5171 non-null   float64
 9   totalScore    4047 non-null   float64
 10  rank          5171 non-null   int64  
 11  imageUrl      5053 non-null   object 
 12  imagesCount   5052 non-null   float64
 13  reviewsCount  5171 non-null   int64  
 14  scrapedAt     5171 non-null   object 
 15  searchString  5171 non-null   object 
dtypes: float64(4), int64(2), object(10)
memory usage: 686.8+ KB


In [148]:
df['scrapedAt'] = pd.to_datetime(df['scrapedAt'])

In [149]:
# Detecta y cuenta filas duplicadas basadas en título, ciudad, calle y coordenadas,
dup = df.duplicated(subset=['title','city','street','location/lat','location/lng']).sum()
dup.sum()

np.int64(1460)

In [150]:
# Quita duplicados según campos clave (título, ciudad, calle y coordenadas), reinicia el índice y guarda una copia limpia del DataFrame.
df = df.loc[~df.duplicated(subset=['title','city','street','location/lat','location/lng'])] \
    .reset_index(drop=True).copy()

In [151]:
# Verifica estructura, cuenta de filas y columnas, tipos y falta de valores tras limpieza de duplicados y modificación de tipo de dato
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3711 entries, 0 to 3710
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   title         3711 non-null   object             
 1   categoryName  3625 non-null   object             
 2   website       1179 non-null   object             
 3   street        3520 non-null   object             
 4   phone         2682 non-null   object             
 5   city          3679 non-null   object             
 6   state         3684 non-null   object             
 7   location/lat  3711 non-null   float64            
 8   location/lng  3711 non-null   float64            
 9   totalScore    2878 non-null   float64            
 10  rank          3711 non-null   int64              
 11  imageUrl      3620 non-null   object             
 12  imagesCount   3619 non-null   float64            
 13  reviewsCount  3711 non-null   int64              
 14  scrapedA

In [152]:
# Limpia y estandariza números internacionales (+54) removiendo dígito “9” extra antes del código local
df['phone'] = df['phone'].astype(str).str.replace(r'\+54\s*9\s*(\d+)', r'+54 \1', regex=True)
df['phone']

0       +54 2281 53-8734
1       +54 2281 41-0748
2       +54 2281 58-4563
3       +54 2281 65-4471
4       +54 2281 50-9728
              ...       
3706    +54 2284 52-8910
3707                 nan
3708    +54 2284 35-7733
3709                 nan
3710    +54 2284 56-1450
Name: phone, Length: 3711, dtype: object

In [153]:
# Estándariza números móviles agregando ‘9’ después del +54 si falta, asegurando el formato internacional correcto.
df['phone'] = df['phone'].str.replace(
    r'^\+54\s*(?!9\s*[\d(])(.+)$', 
    r'+54 9 \1',                   
    regex=True
)
df['phone']

0       +54 9 2281 53-8734
1       +54 9 2281 41-0748
2       +54 9 2281 58-4563
3       +54 9 2281 65-4471
4       +54 9 2281 50-9728
               ...        
3706    +54 9 2284 52-8910
3707                   nan
3708    +54 9 2284 35-7733
3709                   nan
3710    +54 9 2284 56-1450
Name: phone, Length: 3711, dtype: object

In [154]:
# Limpia los nombres de comercios eliminando comillas, asteriscos, signos de interrogación y emoticones.
df['title'] = df['title'].str.replace('"', '', regex=False) \
    .str.replace('*', '', regex=False) \
    .str.replace('?', '', regex=False) \
    .str.replace("'", '', regex=False) \
    .str.replace("👬", '', regex=False) \
    .str.replace("✨", '', regex=False) \
    .str.replace("⚽️", '', regex=False) \
    .str.replace("💖", '', regex=False) \
    .str.replace("♓", '', regex=False) \
    .str.replace("°", '', regex=False) \
    .str.replace("❣️", '', regex=False) \
    .str.replace("🪽", '', regex=False)	  
df.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Me importa un comino,Tienda de alimentos naturales,NaN,Olavarría 1249,+54 9 2281 53-8734,Azul,Provincia de Buenos Aires,-36.791438,-59.854428,4.5,49,https://lh3.googleusercontent.com/p/AF1QipNBGn...,1.0,2,2025-05-19 00:03:04.970000+00:00,Comercio
1,190 grados hierro&madera,Fábrica de muebles,NaN,Lavalle 1372,+54 9 2281 41-0748,Azul,Provincia de Buenos Aires,-36.794246,-59.867392,5.0,16,https://lh3.googleusercontent.com/p/AF1QipMDmE...,14.0,2,2025-06-15 22:57:48.198000+00:00,muebles
2,AGRICOLA AZUL SA,Proveedor de máquinas de metal,NaN,"RN3 km 300,5",+54 9 2281 58-4563,Azul,Provincia de Buenos Aires,-36.802081,-59.852856,4.8,7,https://lh3.googleusercontent.com/gps-cs-s/AC9...,29.0,17,2025-06-15 22:57:22.428000+00:00,maquinaria
3,AGROREPUESTOS AZUL,Servicio agrícola,NaN,RN3 km 301,+54 9 2281 65-4471,Azul,Provincia de Buenos Aires,-36.804447,-59.855641,4.6,55,https://lh3.googleusercontent.com/p/AF1QipMz4V...,16.0,5,2025-06-15 22:57:08.021000+00:00,repuestos
4,ALMÍBAR,Tienda de alimentación,NaN,"Calle Gral. Paz y Comercio, B7300EVX",+54 9 2281 50-9728,Azul,Provincia de Buenos Aires,-36.795313,-59.857420,4.7,3,https://lh3.googleusercontent.com/gps-cs-s/AC9...,16.0,6,2025-06-15 22:56:57.476000+00:00,comercio


In [155]:
# Estandariza title y searchString aplicando formato de título.
df['title'] = df['title'].str.title()
df['searchString'] = df['searchString'].str.title()
df[['title', 'searchString']]

,title,searchString
0,Me Importa Un Comino,Comercio
1,190 Grados Hierro&Madera,Muebles
2,Agricola Azul Sa,Maquinaria
3,Agrorepuestos Azul,Repuestos
4,Almíbar,Comercio
...,...,...
3706,Lucas Herrera Inmobiliaria,Venta
3707,Pizzas Y Empanadas,Venta
3708,Bernardo Erro Inmobiliaria,Venta
3709,Pesca Camping,Venta


In [156]:
# Organiza los datos alfabéticamente por título y muestra los primeros 5 registros.
df = df.sort_values(by='title', ascending=True)
df.head(5)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
3052,. Prix Equipamientos 4X4 Bracco Walker,Tienda de accesorios para automóviles,https://instagram.com/grandprixequipamientos4x...,"Ruta 226, Km 295",+54 9 2284 68-6539,Olavarría,Provincia de Buenos Aires,-36.894279,-60.283588,3.9,17,https://lh3.googleusercontent.com/p/AF1QipMjZI...,13.0,14,2025-05-29 00:56:28.836000+00:00,Repuestos
3503,06 - Alvarez Neumáticos - Olavarría,Tienda de neumáticos,http://www.alvarezneumaticos.com/,Av. Pringles 2776,+54 9 2284 44-5006,Olavarría,Provincia de Buenos Aires,-36.898109,-60.317314,4.7,47,https://lh3.googleusercontent.com/p/AF1QipPpU8...,39.0,292,2025-06-11 18:14:01.809000+00:00,Venta
1047,100%Argentino,Comercio,http://www.100arg.com/,Primera Junta 3055,+54 9 249 444-0813,Tandil,Provincia de Buenos Aires,-37.294857,-59.140034,NaN,5,NaN,NaN,0,2025-05-04 16:08:17.581000+00:00,Comercio
2893,104 Polirrubro,Tienda de alimentación,NaN,Azopardo 1598,+54 9 2284 42-1670,Olavarría,Provincia de Buenos Aires,-36.893022,-60.341844,3.8,5,https://lh3.googleusercontent.com/gps-cs-s/AC9...,18.0,4,2025-05-29 00:56:12.808000+00:00,Comercio
3018,185 Motos-Atv,Concesionario de motos,http://www.facebook.com/185motos,Rivadavia 1940,+54 9 2284 41-6686,Olavarría,Provincia de Buenos Aires,-36.899603,-60.329491,4.6,20,https://lh3.googleusercontent.com/gps-proxy/AL...,4.0,33,2025-05-29 00:56:23.370000+00:00,Venta


In [157]:
# Verifica la cantidad total de registros en la columna 'website', incluyendo duplicados.
df['website'].value_counts().sum()

np.int64(1179)

In [ ]:
# Elimina el prefijo '/url?q=' de las URLs en la columna 'website' y conserva solo la URL base.
df['website'] = df['website'].str.replace(
    r'^\s*/url\?q=(https?://.*)',
    r'\1',
    regex=True
)

In [160]:
df['website'].value_counts()

website
http://www.cooperativaobrera.coop/                                                                                                                                                                           5
http://www.ypf.com/                                                                                                                                                                                          4
https://www.carrefour.com.ar/                                                                                                                                                                                3
https://www.tiendadash.com.ar/                                                                                                                                                                               3
http://www.golopolis.com.ar/                                                                                                                                        

In [161]:
# Se listan las categorias
list = df['categoryName'].unique().tolist()
list

['Tienda de accesorios para automóviles',
 'Tienda de neumáticos',
 'Comercio',
 'Tienda de alimentación',
 'Concesionario de motos',
 'Fábrica de muebles',
 'Fabricante / distribuidor de estatuas',
 'Tienda de ropa de deportes',
 'Tienda de ropa',
 'Construcción de maquinaria',
 'Tienda de ropa de hombre',
 'Tienda de informática',
 'Panadería',
 'Tienda general',
 'Tienda de materiales para la construcción',
 'Tienda de camisetas personalizadas',
 'Carpintero',
 'Frutería',
 'Papelería',
 'Cristalero',
 'Servicio de guardias de seguridad',
 'Taller de reparación de automóviles',
 'Proveedor de repuestos de carrocería de automóviles',
 'Tienda de móviles',
 'Tienda de baterías para automóvil',
 'Tienda de lencería',
 'Colchonería',
 'Tienda de ropa de mujer',
 'Servicio de afilado',
 'Tienda de repuestos para automóviles',
 'Estación de inspección de vehículos',
 'Tienda de lotería',
 'Tienda de cosméticos',
 'Proveedor de máquinas de metal',
 'Fábrica de maquinaria agrícola',
 'Ofici

In [162]:
# Estandariza categorías en la columna 'categoryName' usando un diccionario:
# asigna valores generales como "Comestibles" o "Servicios" según el mapeo definido en categoryNew.
categoryNew = {
    'Tienda de alimentación' :'Comestibles',
    'Tienda general' : 'Comestibles',
    'Panadería' : 'Comestibles',
    'Frutería' : 'Comestibles',
    'Carnicería' : 'Comestibles',
    'Tienda de postres' : 'Comestibles',
    'Granja' : 'Comestibles',
    'Supermercado' : 'Comestibles',
    'Tienda de fiambres' : 'Comestibles',
    'Pizzería' : 'Comestibles',
    'Kiosco' : 'Comestibles',
    'Tienda de alimentos orgánicos' : 'Comestibles',
    'Tienda de golosinas' : 'Comestibles',
    'Pastelería' : 'Comestibles',
    'Restaurante especializado en barbacoa' : 'Comestibles',
    'Tienda de vinos' : 'Comestibles',
    'Cervecería artesanal' : 'Comestibles',
    'Tienda de alimentos naturales' : 'Comestibles',
    'Bar' : 'Comestibles',
    'Cafetería' : 'Comestibles',
    'Tienda de cerveza' : 'Comestibles',
    'Restaurante familiar' : 'Comestibles',
    'Tienda de productos de chocolate' : 'Comestibles',
    'Churrería' : 'Comestibles',
    'Comedor social' : 'Comestibles',
    'Pastelería francesa' : 'Comestibles',
    'Quiosco' : 'Comestibles',
    'Parrilla' : 'Comestibles',
    'Restaurante' : 'Comestibles',
    'Restaurante de comida para llevar' : 'Comestibles',
    'Tienda de comestibles :  periódicos y medicamentos' : 'Comestibles',
    'Heladería' : 'Comestibles',
    'Tienda de artículos para barbacoas' : 'Comestibles',
    'Mayorista de confitería' : 'Comestibles',
    'Supermercado chino' : 'Comestibles',
    'Vinoteca' : 'Comestibles',
    'Tienda naturista' : 'Comestibles',
    'Tienda de bebidas alcohólicas' : 'Comestibles',
    'Quesería' : 'Comestibles',
    'Tienda de productos lácteos' : 'Comestibles',
    'Delicatessen de carnicería' : 'Comestibles',
    'Mercado' : 'Comestibles',
    'Fábrica de conservas' : 'Comestibles',
    'Mercadillo' : 'Comestibles',
    'Proveedor de huevos' : 'Comestibles',
    'Servicio de catering' : 'Comestibles',
    'Delicatessen' : 'Comestibles',
    'Tienda de comestibles mayorista' : 'Comestibles',
    'Almacén' : 'Comestibles',
    'Mercado de alimentos frescos' : 'Comestibles',
    'Pollería' : 'Comestibles',
    'Tienda de frutos secos' : 'Comestibles',
    'Restaurante de burritos' : 'Comestibles',
    'Tienda de jamones' : 'Comestibles',
    'Tienda de pasta' : 'Comestibles',
    'Tienda de artículos para decoración en pastelería' : 'Comestibles',
    'Mayorista' : 'Comestibles',
    'Tienda de artículos de cocina' : 'Comestibles',
    'Tienda de ropa vintage' : 'Ropa y Calzado',
    'Tienda de camisetas' : 'Ropa y Calzado',
    'Tienda de ropa de vestir' : 'Ropa y Calzado',
    'Tienda de equipamiento y ropa para actividades al aire libre' : 'Ropa y Calzado',
    'Tienda de ropa de playa' : 'Ropa y Calzado',
    'Tienda de ropa de deportes' : 'Ropa y Calzado',
    'Tienda de ropa' : 'Ropa y Calzado',
    'Tienda de ropa de hombre' : 'Ropa y Calzado',
    'Tienda de camisetas personalizadas' : 'Ropa y Calzado',
    'Tienda de lencería' : 'Ropa y Calzado',
    'Tienda de ropa de mujer' : 'Ropa y Calzado',
    'Tienda de deportes' : 'Ropa y Calzado',
    'Tienda de ropa del Oeste' : 'Ropa y Calzado',
    'Tienda de equipos de baños' : 'Ropa y Calzado',
    'Tienda de artículos para bebés' : 'Ropa y Calzado',
    'Tienda de ropa infantil' : 'Ropa y Calzado',
    'Mercería' : 'Ropa y Calzado',
    'Tienda de ropa para bebés' : 'Ropa y Calzado',
    'Tienda de ropa de trabajo' : 'Ropa y Calzado',
    'Tienda de uniformes' : 'Ropa y Calzado',
    'Tienda de lana' : 'Ropa y Calzado',
    'Tienda de artículos de natación' : 'Ropa y Calzado',
    'Zapatería' : 'Ropa y Calzado',
    'Tintorería' : 'Ropa y Calzado',
    'Tienda de ropa de segunda mano' : 'Ropa y Calzado',
    'Tienda de ropa interior' : 'Ropa y Calzado',
    'Tienda de ropa para jóvenes' : 'Ropa y Calzado',
    'Tienda de arreglos de ropa' : 'Ropa y Calzado',
    'Tienda de botas' : 'Ropa y Calzado',
    'Outlet' : 'Ropa y Calzado',
    'Servicio de reparación de parabrisas y pantallas' : 'Vehiculos y Transporte',
    'Concesionario Toyota' : 'Vehiculos y Transporte',
    'Empresa de transporte por camión' : 'Vehiculos y Transporte',
    'Servicio de transporte' : 'Vehiculos y Transporte',
    'Fabricante de repuestos para automóviles' : 'Vehiculos y Transporte',
    'Servicio de remolque' : 'Vehiculos y Transporte',
    'Servicio de taxis' : 'Vehiculos y Transporte',
    'Taller de reparación de motores diésel' : 'Vehiculos y Transporte',
    'Servicio de restauración de automóviles' : 'Vehiculos y Transporte',
    'Taller de reparación de motos' : 'Vehiculos y Transporte',
    'Servicio de lavado a presión' : 'Vehiculos y Transporte',
    'Servicio de lavado de coches' : 'Vehiculos y Transporte',
    'Tienda de accesorios para automóviles' : 'Vehiculos y Transporte',
    'Tienda de neumáticos' : 'Vehiculos y Transporte',
    'Concesionario de motos' : 'Vehiculos y Transporte',
    'Tienda de piezas de automóvil' : 'Vehiculos y Transporte',
    'Pintura de automóviles' : 'Vehiculos y Transporte',
    'Taller de reparación de automóviles' : 'Vehiculos y Transporte',
    'Proveedor de repuestos de carrocería de automóviles' : 'Vehiculos y Transporte',
    'Tienda de baterías para automóvil' : 'Vehiculos y Transporte',
    'Tienda de repuestos para automóviles' : 'Vehiculos y Transporte',
    'Taller mecánico' : 'Vehiculos y Transporte',
    'Estación de inspección de vehículos' : 'Vehiculos y Transporte',
    'Compraventa de automóviles' : 'Vehiculos y Transporte',
    'Concesionario Chevrolet' : 'Vehiculos y Transporte',
    'Tienda de accesorios para camiones' : 'Vehiculos y Transporte',
    'Concesionario de remolques' : 'Vehiculos y Transporte',
    'Gasolinera' : 'Vehiculos y Transporte',
    'Tienda de repuestos de automóviles usados' : 'Vehiculos y Transporte',
    'Tienda de repuestos para motos' : 'Vehiculos y Transporte',
    'Parada para camiones' : 'Vehiculos y Transporte',
    'Desguace' : 'Vehiculos y Transporte',
    'Concesionario de automóviles' : 'Vehiculos y Transporte',
    'Aparcamiento' : 'Vehiculos y Transporte',
    'Servicio de distribución' : 'Vehiculos y Transporte',
    'Concesionario de autocaravanas' : 'Vehiculos y Transporte',
    'Concesionario de tractores' : 'Vehiculos y Transporte',
    'Concesionario Volkswagen' : 'Vehiculos y Transporte',
    'Taller de automóviles' : 'Vehiculos y Transporte',
    'Taller de camiones' : 'Vehiculos y Transporte',
    'Taller de chapa y pintura' : 'Vehiculos y Transporte',
    'Taller de frenos' : 'Vehiculos y Transporte',
    'Taller de revisión de automóviles' : 'Vehiculos y Transporte',
    'Tienda de vehículos de segunda mano y de ocasión' : 'Vehiculos y Transporte',
    'Tienda de repuestos para coches de carreras' : 'Vehiculos y Transporte',
    'Tienda de neumáticos usados' : 'Vehiculos y Transporte',
    'Tienda de motocicletas' : 'Vehiculos y Transporte',
    'Tienda de equipos estéreo para automóviles' : 'Vehiculos y Transporte',
    'Tienda de baterías' : 'Vehiculos y Transporte',
    'Servicio de transporte de mercancías' : 'Vehiculos y Transporte',
    'Taller de bicicletas' : 'Vehiculos y Transporte',
    'Servicio de cambio de aceite' : 'Vehiculos y Transporte',
    'Tienda de bicicletas' : 'Vehiculos y Transporte',
    'Tienda de software' : 'Tecnologia',
    'Tienda de informática' : 'Tecnologia',
    'Tienda de móviles' : 'Tecnologia',
    'Diseñador de sitios web' : 'Tecnologia',
    'Establecimiento de reparación de artículos electrónicos' : 'Tecnologia',
    'Tienda de electrónica' : 'Tecnologia',
    'Empresa de alojamiento web' : 'Tecnologia',
    'Servicio de informática' : 'Tecnologia',
    'Proveedor de servicios de Internet' : 'Tecnologia',
    'Consultora informática' : 'Tecnologia',
    'Tienda de reparación de teléfonos móviles' : 'Tecnologia',
    'Tienda de ordenadores de segunda mano' : 'Tecnologia',
    'Tienda de accesorios para móviles' : 'Tecnologia',
    'Tienda de componentes electrónicos' : 'Tecnologia',
    'Soporte y servicios informáticos' : 'Tecnologia',
    'Televisión de circuito cerrado' : 'Tecnologia',
    'Repair service' : 'Tecnologia',
    'Proveedor de servicios de telecomunicaciones' : 'Tecnologia',
    'Proveedor de sistemas de seguridad' : 'Tecnologia',
    'Proveedor de repuestos de electrodomésticos' : 'Tecnologia',
    'Empresa de software' : 'Tecnologia',
    'Distribuidor de equipos de telecomunicación' : 'Tecnologia',
    'Proveedor de equipos audiovisuales' : 'Tecnologia',
    'Corredor de seguros' : 'Profesionales',
    'Abogado' : 'Profesionales',
    'Apicultor' : 'Profesionales',
    'Asesor' : 'Profesionales',
    'Soldador' : 'Profesionales',
    'Servicios legales' : 'Profesionales',
    'Agencia de publicidad' : 'Profesionales',
    'Contable' : 'Profesionales',
    'Dentista cosmético' : 'Profesionales',
    'Fotógrafo' : 'Profesionales',
    'Ingeniero' : 'Profesionales',
    'Masajista' : 'Profesionales',
    'Persona de mantenimiento' : 'Profesionales',
    'Pintor' : 'Profesionales',
    'Proveedor de servicios de asistencia jurídica' : 'Profesionales',
    'Psicólogo' : 'Profesionales',
    'Tienda de audífonos' : 'Profesionales',
    'Tienda de artículos ortopédicos' : 'Profesionales',
    'Modista' : 'Profesionales',
    'Numismático' : 'Profesionales',
    'Óptica' : 'Profesionales',
    'Diseñador gráfico' : 'Profesionales',
    'Tienda de suministros para odontología' : 'Profesionales',
    'Agencia de marketing' : 'Profesionales',
    'Tienda de belleza y salud' : 'Belleza y Salud',
    'Salón de manicura y pedicura' : 'Belleza y Salud',
    'Academia de estética' : 'Belleza y Salud',
    'Tienda de cosméticos' : 'Belleza y Salud',
    'Tienda de productos de belleza' : 'Belleza y Salud',
    'Peluquería' : 'Belleza y Salud',
    'Centro de estética' : 'Belleza y Salud',
    'Boutique' : 'Belleza y Salud',
    'Esteticista' : 'Belleza y Salud',
    'Perfumería' : 'Belleza y Salud',
    'Tienda de accesorios de moda' : 'Belleza y Salud',
    'Tienda de artículos de aseo personal' : 'Belleza y Salud',
    'Cabina de teléfono público' : 'Otras Categorias',
    'Centro artístico' : 'Otras Categorias',
    'Fabricante / distribuidor de estatuas' : 'Otras Categorias',
    'Oficinas de empresa' : 'Otras Categorias',
    'Papelería' : 'Otras Categorias',
    'Proveedor de artículos de cuero' : 'Otras Categorias',
    'Proveedor de productos de caucho' : 'Otras Categorias',
    'Taller de costura' : 'Otras Categorias',
    'Tienda de alarmas antirrobo' : 'Otras Categorias',
    'Tienda de aromaterapia' : 'Otras Categorias',
    'Tienda de artículos de golf' : 'Otras Categorias',
    'Tienda de artículos para correr' : 'Otras Categorias',
    'Tienda de artículos religiosos' : 'Otras Categorias',
    'Tienda de bolígrafos' : 'Otras Categorias',
    'Tienda de bricolaje' : 'Otras Categorias',
    'Tienda de CDs' : 'Otras Categorias',
    'Tienda de costura' : 'Otras Categorias',
    'Tienda de deportes al aire libre' : 'Otras Categorias',
    'Tienda de lotería' : 'Otras Categorias',
    'Tienda de manualidades y bellas artes' : 'Otras Categorias',
    'Tienda de material deportivo' : 'Otras Categorias',
    'Tienda de material escolar' : 'Otras Categorias',
    'Tienda de pancartas publicitarias' : 'Otras Categorias',
    'Tienda de suministros para hockey' : 'Otras Categorias',
    'Tienda de suministros para serigrafía' : 'Otras Categorias',
    'Tienda de telas' : 'Otras Categorias',
    'Agencia de bienes inmuebles comerciales' : 'Inmobiliario',
    'Agentes inmobiliarios' : 'Inmobiliario',
    'Alojamiento con servicio' : 'Inmobiliario',
    'Agencia inmobiliaria' : 'Inmobiliario',
    'Consultor inmobiliario' : 'Inmobiliario',
    'Sociedad de viviendas' : 'Inmobiliario',
    'Casa rural' : 'Inmobiliario',
    'Edificio de apartamentos' : 'Inmobiliario',
    'Inmobiliaria' : 'Inmobiliario',
    'Centro comercial outlet' : 'Comercio',
    'Centro comercial' : 'Comercio',
    'Farmacia' : 'Comercio',
    'Ferretería' : 'Comercio',
    'Floristería' : 'Comercio',
    'Comercio' : 'Comercio',
    'Imprenta' : 'Comercio',
    'Lavandería' : 'Comercio',
    'Librería' : 'Comercio',
    'Locutorio' : 'Comercio',
    'Empresa de medios de comunicación' : 'Comercio',
    'Empresa de mensajería' : 'Comercio',
    'Estudio de grabación' : 'Comercio',
    'Estudio de retratos' : 'Comercio',
    'Estudio de tatuajes' : 'Comercio',
    'Cerería' : 'Comercio',
    'Copistería' : 'Comercio',
    'Tienda de instrumentos musicales' : 'Entretenimiento',
    'Tienda de juegos' : 'Entretenimiento',
    'Centro de comercio y exhibiciones' : 'Entretenimiento',
    'Centro de deportes de aventura' : 'Entretenimiento',
    'Cibercafé' : 'Entretenimiento',
    'Cine' : 'Entretenimiento',
    'Empresa de organización de eventos' : 'Entretenimiento',
    'Juguetería' : 'Entretenimiento',
    'Servicio de fiestas infantiles' : 'Entretenimiento',
    'Tienda de disfraces' : 'Entretenimiento',
    'Tienda de coleccionables' : 'Entretenimiento',
    'Tienda de cómics' : 'Entretenimiento',
    'Tienda de artículos de fiesta' : 'Entretenimiento',
    'Agencia de préstamos' : 'Financiero',
    'Banco' : 'Financiero',
    'Cobro de deudas' : 'Financiero',
    'Centro de negocios' : 'Financiero',
    'Institución financiera' : 'Financiero',
    'Agencia de alquiler de cabañas' : 'Turismo',
    'Agencia de viajes' : 'Turismo',
    'Agencia de visitas turísticas' : 'Turismo',
    'Cabañas de madera' : 'Turismo',
    'Armería' : 'Caza y Pesca',
    'Tienda de artículos de caza' : 'Caza y Pesca',
    'Tienda de artículos de pesca' : 'Caza y Pesca',
    'Tienda de artículos de camping' : 'Caza y Pesca',
    'Tienda de cebos' : 'Caza y Pesca',
    'Colchonería' : 'Hogar y Muebles',
    'Tienda de colchas' : 'Hogar y Muebles',
    'Tienda de ropa de hogar' : 'Hogar y Muebles',
    'Fábrica de muebles' : 'Hogar y Muebles',
    'Tienda de muebles' : 'Hogar y Muebles',
    'Tienda especializada en camas' : 'Hogar y Muebles',
    'Tienda de sofás' : 'Hogar y Muebles',
    'Tienda de ropa de cama' : 'Hogar y Muebles',
    'Tienda de muebles antiguos' : 'Hogar y Muebles',
    'Tienda de muebles de cocina' : 'Hogar y Muebles',
    'Tienda de muebles de exterior' : 'Hogar y Muebles',
    'Tienda de muebles de pino' : 'Hogar y Muebles',
    'Tienda de muebles de segunda mano' : 'Hogar y Muebles',
    'Tienda de mobiliario de oficina' : 'Hogar y Muebles',
    'Tienda de espejos' : 'Hogar y Muebles',
    'Tienda de alfombras' : 'Hogar y Muebles',
    'Supermercado de muebles' : 'Hogar y Muebles',
    'Tienda de electrodomésticos' : 'Hogar y Muebles',
    'Tienda de cortinas' : 'Hogar y Muebles',
    'Tapicería' : 'Hogar y Muebles',
    'Tienda de equipos de sonido' : 'Hogar y Muebles',
    'Proveedor de productos de limpieza' : 'Hogar y Muebles',
    'Proveedor de equipos de oficina' : 'Hogar y Muebles',
    'Tienda aire acondicionado' : 'Hogar y Muebles',
    'Tienda de material de oficina' : 'Hogar y Muebles',
    'Tienda de fotografía' : 'Hogar y Muebles',
    'Servicio de limpieza por chorro de arena' : 'Agricola',
    'Distribuidores de carretillas elevadoras' : 'Agricola',
    'Construcción de maquinaria' : 'Agricola',
    'Proveedor de máquinas de metal' : 'Agricola',
    'Fábrica de maquinaria agrícola' : 'Agricola',
    'Taller de maquinaria' : 'Agricola',
    'Fábrica' : 'Agricola',
    'Tienda de herramientas' : 'Agricola',
    'Servicio de alquiler de herramientas' : 'Agricola',
    'Fábrica de maquinaria' : 'Agricola',
    'Tienda de productos agrícolas' : 'Agricola',
    'Tienda de aves de corral' : 'Agricola',
    'Proveedor de semillas' : 'Agricola',
    'Empresa de fumigación y control de plagas' : 'Agricola',
    'Servicio de reparación de maquinaria agrícola' : 'Maquinaria',
    'Ingeniero agrónomo' : 'Maquinaria',
    'Servicio agrícola' : 'Maquinaria',
    'Mercado de productos agrícolas' : 'Maquinaria',
    'Proveedor de equipos agrícolas' : 'Maquinaria',
    'Centro de jardinería' : 'Maquinaria',
    'Agencia de alquiler de grúas' : 'Maquinaria',
    'Taller de metalurgia' : 'Maquinaria',
    'Taller de reparación de herramientas' : 'Maquinaria',
    'Tienda de suministros para soldadura' : 'Maquinaria',
    'Herrero' : 'Maquinaria',
    'Fundición' : 'Maquinaria',
    'Proveedor de equipos para fábricas' : 'Maquinaria',
    'Fabricante de repuestos para maquinaria' : 'Maquinaria',
    'Equipos para panaderías' : 'Maquinaria',
    'Empresa de fabricación de plástico' : 'Maquinaria',
    'Fábrica de bolsas de plástico' : 'Maquinaria',
    'Fábrica de papel' : 'Maquinaria',
    'Chatarrería' : 'Maquinaria',
    'Tienda de materiales para la construcción' : 'Construccion',
    'Tienda de artículos para pintura' : 'Construccion',
    'Empresa constructora' : 'Construccion',
    'Condominio' : 'Construccion',
    'Constructor de plataformas y terrazas' : 'Construccion',
    'Constructor' : 'Construccion',
    'Ventana de aluminio' : 'Construccion',
    'Tienda de persianas' : 'Construccion',
    'Tienda de pinturas' : 'Construccion',
    'Urbanización' : 'Construccion',
    'Vivero' : 'Construccion',
    'Tienda de materiales para suelos' : 'Construccion',
    'Tienda de iluminación' : 'Construccion',
    'Tienda de cristales' : 'Construccion',
    'Tienda de electricidad' : 'Construccion',
    'Tienda de cerámica' : 'Construccion',
    'Tienda de azulejos' : 'Construccion',
    'Tienda de accesorios para ventanas' : 'Construccion',
    'Contratista de aire acondicionado' : 'Construccion',
    'Contratista de albañilería' : 'Construccion',
    'Contratista de excavación' : 'Construccion',
    'Contratista de materiales para suelos' : 'Construccion',
    'Contratista de revestimientos' : 'Construccion',
    'Contratista' : 'Construccion',
    'Carpintería' : 'Construccion',
    'Electricista' : 'Construccion',
    'Empresa de acabados metálicos' : 'Construccion',
    'Empresa de calefacción' : 'Construccion',
    'Empresa de climatización' : 'Construccion',
    'Empresa de construcción' : 'Construccion',
    'Establecimiento de venta de madera' : 'Construccion',
    'Fábrica de puertas' : 'Construccion',
    'Interiorista' : 'Construccion',
    'Jardinero' : 'Construccion',
    'Pintura' : 'Construccion',
    'Proveedor de maquinaria de construcción' : 'Construccion',
    'Proveedor de materiales de construcción' : 'Construccion',
    'Tienda de artículos relacionados con el gas' : 'Construccion',
    'Tienda de artículos de fontanería' : 'Construccion',
    'Parque' : 'Construccion',
    'Piscina' : 'Construccion',
    'Instalador de gas' : 'Construccion',
    'Proveedor de equipos de energía solar' : 'Construccion',
    'Carpintero' : 'Construccion',
    'Arquitecto' : 'Construccion',
    'Cristalero' : 'Construccion',
    'Proveedor de equipos de calefacción' : 'Construccion',
    'Tienda de estufas de leña' : 'Construccion',
    'Bazar' : 'Bazar y Regaleria',
    'Tienda de artesanías' : 'Bazar y Regaleria',
    'Tienda de antigüedades' : 'Bazar y Regaleria',
    'Marroquinería' : 'Bazar y Regaleria',
    'Tienda de cubertería' : 'Bazar y Regaleria',
    'Tienda de artículos para el hogar' : 'Bazar y Regaleria',
    'Tienda de regalos' : 'Bazar y Regaleria',
    'Tienda de recuerdos' : 'Bazar y Regaleria',
    'Artesanía' : 'Bazar y Regaleria',
    'Cuchillería' : 'Bazar y Regaleria',
    'Joyería' : 'Bazar y Regaleria',
    'Grabador de joyas' : 'Bazar y Regaleria',
    'Veterinario' : 'Vet y Petshop',
    'Tienda de productos para mascotas' : 'Vet y Petshop',
    'Tienda de alimentos para animales' : 'Vet y Petshop',
    'Tienda de animales' : 'Vet y Petshop',
    'Hospital veterinario' : 'Vet y Petshop',
    'Asociación u organización' : 'Asociaciones',
    'Cámara de comercio' : 'Asociaciones',
    'Casa de subastas' : 'Asociaciones',
    'Catastro' : 'Asociaciones',
    'Comunidad para jubilados' : 'Asociaciones',
    'Cuarteles militares' : 'Asociaciones',
    'Centro de formación' : 'Asociaciones',
    'Oficina de correos' : 'Asociaciones',
    'Oficina de la Administración' : 'Asociaciones',
    'Registro civil' : 'Asociaciones',
    'Delegación de Servicios Sociales' : 'Asociaciones',
    'Departamento de obras públicas' : 'Asociaciones',
    'Departamento de salud pública' : 'Asociaciones',
    'Sindicato' : 'Asociaciones',
    'Servicio de administración de empresas' : 'Asociaciones',
    'Policía nacional' : 'Asociaciones',
    'Hospital' : 'Asociaciones',
    'Instituto de investigación' : 'Asociaciones',
    'Empresa suministradora de gas butano' : 'Asociaciones',
    'Organización de servicios sociales' : 'Asociaciones',
    'Servicio de conserjería' : 'Servicios',
    'Hotel' : 'Servicios',
    'Servicio de afilado' : 'Servicios',
    'Servicio de estampado en relieve' : 'Servicios',
    'Servicio de guardias de seguridad' : 'Servicios',
    'Compañía de seguros' : 'Servicios',
    'Compañía del gas' : 'Servicios',
    'Compañía telefónica' : 'Servicios',
    'Servicio de impermeabilización' :'Servicios',
    'Servicio de limpieza' : 'Servicios',
    'Servicio de limpieza de alfombras' : 'Servicios',
    'Servicio de limpieza de tapicería' : 'Servicios',
    'Servicio de marketing por Internet' : 'Servicios',
    'Servicio de mudanzas' : 'Servicios',
    'Servicio de mudanzas y almacenamiento' : 'Servicios',
    'Servicio de paseo a caballo' : 'Servicios',
    'Servicio de pañales' : 'Servicios',
    'Servicio de polarización de ventanas' : 'Servicios',
    'Servicio de reacondicionamiento de motores' : 'Servicios',
    'Servicio de reparación de aire acondicionado' : 'Servicios',
    'Servicio de reparación de compresores de aire' : 'Servicios',
    'Servicio de reparación de electrodomésticos' : 'Servicios',
    'Servicio de reparación de impresoras' : 'Servicios',
    'Servicio de reparación de lavadoras y secadoras' : 'Servicios',
    'Servicio de reparación de lámparas' : 'Servicios',
    'Servicio de reparación de máquinas de coser' : 'Servicios',
    'Servicio de reparación de ordenadores' : 'Servicios',
    'Servicio de reparación de sistemas eléctricos para automóviles' : 'Servicios',
    'Servicio de reparación de sistemas hidráulicos' : 'Servicios',
    'Servicio de reparación de televisores' : 'Servicios',
    'Servicio de rescate de animales' :  'Servicios',
    'Servicio de seguridad contra incendios' : 'Servicios',
    'Servicio de tecnología de eventos' : 'Tecnologia',
    'Servicio de transferencias de dinero' : 'Financiero',
    'Servicio forestal' : 'Servicios',
    'Servicios de diseño' :  'Servicios',
    'Servicios de documentación' :  'Servicios',
    'Servicios de limpieza' :  'Servicios',
    'Tienda de comestibles, periódicos y medicamentos' : 'Comestibles',
    'Tienda de señalización' : 'Otras Categorias',
    'Tienda de videojuegos' : 'Entretenimiento',
    'Tienda infantil' : 'Entretenimiento',
    'Videoclub' : 'Entretenimiento',
    'Planta de tratamiento de aguas residuales' : 'Servicios'
}
df['categoryName'] = df['categoryName'].replace(categoryNew)

In [163]:
# Verifica la distribución de categorías tras la limpieza
df['categoryName'].value_counts()

categoryName
Comestibles               901
Ropa y Calzado            588
Vehiculos y Transporte    407
Comercio                  380
Construccion              197
Tecnologia                195
Hogar y Muebles           163
Inmobiliario              156
Bazar y Regaleria         144
Otras Categorias           71
Agricola                   66
Servicios                  65
Maquinaria                 53
Belleza y Salud            51
Profesionales              37
Entretenimiento            36
Vet y Petshop              32
Asociaciones               24
Caza y Pesca               14
Turismo                    14
Financiero                 14
Servicio de lavandería     10
Tienda de delicatessen      5
Servicio de jardinería      2
Name: count, dtype: int64

In [164]:
# Se listan las categorías que quedaron definidas
df['categoryName'].unique().tolist()

['Vehiculos y Transporte',
 'Comercio',
 'Comestibles',
 'Hogar y Muebles',
 'Otras Categorias',
 'Ropa y Calzado',
 'Agricola',
 'Tecnologia',
 'Construccion',
 'Servicios',
 'Belleza y Salud',
 'Maquinaria',
 nan,
 'Inmobiliario',
 'Bazar y Regaleria',
 'Turismo',
 'Caza y Pesca',
 'Asociaciones',
 'Profesionales',
 'Financiero',
 'Vet y Petshop',
 'Servicio de lavandería',
 'Entretenimiento',
 'Tienda de delicatessen',
 'Servicio de jardinería']

In [ ]:
# Define qué strings deben ser tratados como nulos en las columnas seleccionadas.
nulls = ['', ' ', 'N/A', 'nan', 'NaN', 'None']

# Define las columnas a las que quieres aplicar este proceso.
col = ['title', 'categoryName', 'website', 'street', 'phone', 'city', 'state', 'imageUrl']

# Itera sobre cada columna y aplica la lógica.
for col in col:
    df.loc[:, col] = df[col].replace(nulls, np.nan) # Estandariza a np.nan
    df.loc[:, col] = df[col].fillna('incomplete')

In [ ]:
# Define qué strings deben ser tratados como nulos en las columnas seleccionadas.
nulls = ['', ' ', 'N/A', 'nan', 'NaN', 'None']

# Define las columnas a las que se aplica este proceso.
colnum = ['location/lat' ,'location/lng', 'totalScore', 'imagesCount', 'rank', 'reviewsCount']

# Itera sobre cada columna y aplica la lógica.
for colnum in colnum:
    df.loc[:, colnum] = df[colnum].replace(nulls, np.nan) # Estandariza a np.nan
    df.loc[:, colnum] = df[colnum].fillna(0)

In [ ]:
# 3711 entries sin valores nulos en las columnas
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3711 entries, 3052 to 2047
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   title         3711 non-null   object             
 1   categoryName  3711 non-null   object             
 2   website       3711 non-null   object             
 3   street        3711 non-null   object             
 4   phone         3711 non-null   object             
 5   city          3711 non-null   object             
 6   state         3711 non-null   object             
 7   location/lat  3711 non-null   float64            
 8   location/lng  3711 non-null   float64            
 9   totalScore    3711 non-null   float64            
 10  rank          3711 non-null   int64              
 11  imageUrl      3711 non-null   object             
 12  imagesCount   3711 non-null   float64            
 13  reviewsCount  3711 non-null   int64              
 14  scrapedAt 

In [168]:
df.head(10)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
3052,. Prix Equipamientos 4X4 Bracco Walker,Vehiculos y Transporte,https://instagram.com/grandprixequipamientos4x...,"Ruta 226, Km 295",+54 9 2284 68-6539,Olavarría,Provincia de Buenos Aires,-36.894279,-60.283588,3.9,17,https://lh3.googleusercontent.com/p/AF1QipMjZI...,13.0,14,2025-05-29 00:56:28.836000+00:00,Repuestos
3503,06 - Alvarez Neumáticos - Olavarría,Vehiculos y Transporte,http://www.alvarezneumaticos.com/,Av. Pringles 2776,+54 9 2284 44-5006,Olavarría,Provincia de Buenos Aires,-36.898109,-60.317314,4.7,47,https://lh3.googleusercontent.com/p/AF1QipPpU8...,39.0,292,2025-06-11 18:14:01.809000+00:00,Venta
1047,100%Argentino,Comercio,http://www.100arg.com/,Primera Junta 3055,+54 9 249 444-0813,Tandil,Provincia de Buenos Aires,-37.294857,-59.140034,0.0,5,incomplete,0.0,0,2025-05-04 16:08:17.581000+00:00,Comercio
2893,104 Polirrubro,Comestibles,incomplete,Azopardo 1598,+54 9 2284 42-1670,Olavarría,Provincia de Buenos Aires,-36.893022,-60.341844,3.8,5,https://lh3.googleusercontent.com/gps-cs-s/AC9...,18.0,4,2025-05-29 00:56:12.808000+00:00,Comercio
3018,185 Motos-Atv,Vehiculos y Transporte,http://www.facebook.com/185motos,Rivadavia 1940,+54 9 2284 41-6686,Olavarría,Provincia de Buenos Aires,-36.899603,-60.329491,4.6,20,https://lh3.googleusercontent.com/gps-proxy/AL...,4.0,33,2025-05-29 00:56:23.370000+00:00,Venta
1,190 Grados Hierro&Madera,Hogar y Muebles,incomplete,Lavalle 1372,+54 9 2281 41-0748,Azul,Provincia de Buenos Aires,-36.794246,-59.867392,5.0,16,https://lh3.googleusercontent.com/p/AF1QipMDmE...,14.0,2,2025-06-15 22:57:48.198000+00:00,Muebles
2607,3Dimensiones Tandil,Otras Categorias,https://www.mercadolibre.com.ar/pagina/3dimens...,San Martín 1050,+54 9 249 460-6963,Tandil,Provincia de Buenos Aires,-37.323638,-59.130065,5.0,44,https://lh3.googleusercontent.com/p/AF1QipOkvP...,26.0,3,2025-05-29 00:18:03.204000+00:00,Online
3461,42K Bikes,Ropa y Calzado,https://www.facebook.com/42Kolavarria/,Av. del Valle 4444,+54 9 2284 33-3707,Olavarría,Provincia de Buenos Aires,-36.909027,-60.313989,4.9,33,https://lh3.googleusercontent.com/gps-cs-s/AC9...,43.0,52,2025-06-11 18:13:59.910000+00:00,Tienda
2128,47 Street,Ropa y Calzado,http://www.47street.com.ar/,San Martín 685,incomplete,Tandil,Provincia de Buenos Aires,-37.326043,-59.135508,4.7,2,https://lh3.googleusercontent.com/p/AF1QipM2AZ...,15.0,36,2025-05-29 00:14:38.679000+00:00,Ropa
890,4En1 Cras Revocadoras Siberia Revoque Proyectado,Agricola,incomplete,RN226 y 30,+54 9 249 451-3568,Tandil,Provincia de Buenos Aires,-37.289163,-59.169562,5.0,18,https://lh3.googleusercontent.com/p/AF1QipOjsc...,40.0,1,2025-06-15 23:38:59.961000+00:00,Maquinaria


In [169]:
# Se guarda en un archivo final .CSV
df.to_csv('../data/processed/03_regional_Data/regional_Data.csv', index=False)